In [3]:
# Create the connection outside so you don't open/close repeatedly
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=PRATIKSHARAUT\\SQLEXPRESS;'
    'DATABASE=inventory;'
    'Trusted_Connection=yes;'
)
cursor = conn.cursor()

NameError: name 'pyodbc' is not defined

In [131]:

 

import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from datetime import datetime
from tkinter import *
from tkcalendar import DateEntry
from datetime import datetime
from collections import Counter
import sqlite3
import os


def Billing_Entry_UI():
    global current_row
    global all_entry_rows
    global popup_window
    global lp_gross_price_var
    current_row=0
    root = tk.Tk()
    root.title("Billing Management System")
    root.geometry("1400x750")
    root.configure(bg="#ecf0f1")

    # ----------------- Styling -----------------
    style = ttk.Style()
    style.theme_use("clam")

    style.configure("TLabel", font=("Segoe UI", 10), background="#2c3e50", foreground="white")
    style.configure("Header.TLabel", font=("Segoe UI", 20, "bold"), background="#2c3e50", foreground="white")
    style.configure("TButton", font=("Segoe UI", 10), background="#3498db", foreground="white")
    style.configure("TEntry", font=("Segoe UI", 10))
    style.configure("Treeview.Heading", font=("Segoe UI", 10, "bold"), background="#34495e", foreground="white")
    style.configure("Treeview", font=("Segoe UI", 10), rowheight=30)

    # ----------------- Top Header -----------------
    top_frame = tk.Frame(root, bg="#2c3e50", height=60)
    top_frame.pack(side="top", fill="x")

    back_btn = ttk.Button(top_frame, text="← Back")
    back_btn.place(x=10, y=15)

    title_lbl = ttk.Label(top_frame, text="Billing Entry", style="Header.TLabel")
    title_lbl.pack(pady=10)

    # ----------------- Customer Details -----------------
    name_var = tk.StringVar()
    phone_var = tk.StringVar()
    billno_var = tk.StringVar()
    address_var = tk.StringVar()

    cust_frame = tk.LabelFrame(root, text="Customer Details", font=("Segoe UI", 15, "bold"),
                               bg="#2c3e50", fg="white", padx=10, pady=10)
    cust_frame.pack(fill="x", padx=10, pady=5)

    # Labels & Entry Fields
    tk.Label(cust_frame, text="Customer Name:", bg="#2c3e50", fg="white", font=("Segoe UI", 15,"bold")).grid(row=0, column=0, sticky="w", padx=5, pady=5)
    entry_name = ttk.Entry(cust_frame, width=30)
    entry_name.grid(row=0, column=1, padx=5)

    # Your existing fetch_customer_details with minor adjustment
    def fetch_customer_details():
        billno = billno_var.get().strip()
        customer_name = name_var.get().strip()

        if not billno and not customer_name:
            messagebox.showwarning("Input Error", "Please enter either Bill Number or Customer Name")
            return

        try:
            found = False

            if billno:
                cursor.execute("SELECT CustomerName, PhoneNo, Address FROM Customers WHERE BillNumber = ?", (billno,))
                row = cursor.fetchone()
                if row:
                    name_var.set(row[0])
                    phone_var.set(row[1])
                    address_var.set(row[2])
                    found = True

            if not found and customer_name:
                cursor.execute("SELECT BillNumber, PhoneNo, Address FROM Customers WHERE LOWER(CustomerName) LIKE ?", (f"%{customer_name.lower()}%",))
                row = cursor.fetchone()
                if row:
                    billno_var.set(row[0])
                    phone_var.set(row[1])
                    address_var.set(row[2])
                    name_var.set(customer_name)
                    found = True

            if not found:
                messagebox.showerror("Not Found", "No customer found with the given details.")
                billno_var.set("")
                name_var.set("")
                phone_var.set("")
                address_var.set("")

        except Exception as e:
            messagebox.showerror("Database Error", f"Error fetching data: {e}")


    # --- Live Search Popup for Customer Name ---
    def show_customer_suggestions(event):
        typed = name_var.get().strip()
        if not typed:
            if hasattr(root, 'cust_popup'):
                root.cust_popup.destroy()
            return

        try:
            # Search customers starting with typed letters (case-insensitive)
            cursor.execute("SELECT CustomerName FROM Customers WHERE LOWER(CustomerName) LIKE ? ORDER BY CustomerName ASC", (typed.lower() + '%',))
            results = cursor.fetchall()
            names = [row[0] for row in results]

            if not names:
                if hasattr(root, 'cust_popup'):
                    root.cust_popup.destroy()
                return

            # Destroy old popup if exists
            if hasattr(root, 'cust_popup'):
                root.cust_popup.destroy()

            popup = tk.Toplevel(root)
            root.cust_popup = popup
            popup.wm_overrideredirect(True)  # No window decorations
            x = name_entry.winfo_rootx()
            y = name_entry.winfo_rooty() + name_entry.winfo_height()
            popup.geometry(f"+{x}+{y}")

            listbox = tk.Listbox(popup, font=("arial", 12), width=30, height=min(len(names), 10))
            listbox.pack()

            for name in names:
                listbox.insert(tk.END, name)

            def on_select(event):
                if listbox.curselection():
                    selected_name = listbox.get(listbox.curselection())
                    name_var.set(selected_name)

                    # Fetch details for selected customer name
                    try:
                        cursor.execute("SELECT BillNumber, PhoneNo, Address FROM Customers WHERE CustomerName = ?", (selected_name,))
                        row = cursor.fetchone()
                        if row:
                            billno_var.set(row[0])
                            phone_var.set(row[1])
                            address_var.set(row[2])
                        else:
                            billno_var.set("")
                            phone_var.set("")
                            address_var.set("")
                    except Exception as e:
                        messagebox.showerror("Database Error", f"Error fetching data: {e}")

                    popup.destroy()
                    name_entry.focus_set()

            listbox.bind("<<ListboxSelect>>", on_select)

            # Close popup when focus lost
            popup.bind("<FocusOut>", lambda e: popup.destroy())

        except Exception as e:
            if hasattr(root, 'cust_popup'):
                root.cust_popup.destroy()

    name_var = tk.StringVar()
    # Define name_entry BEFORE you bind events to it
    name_entry = Entry(cust_frame, font=("arial", 15), bd=3, relief=GROOVE, textvariable=name_var)
    name_entry.grid(row=0, column=1, padx=10, pady=5)
    
    # Then bind the event after name_entry is defined
    name_entry.bind("<KeyRelease>", show_customer_suggestions)


    # ---------- Search Button ----------
    tk.Button(cust_frame, text="Search", font=("Arial", 12, "bold"), bg="white", fg="#0f4d7d",
              command=fetch_customer_details).grid(row=0, column=6, padx=(5, 10), pady=5)

    # ---------- Cash Discount Section Frame (to keep items aligned together) ----------
    cash_disc_frame = tk.Frame(cust_frame, bg="#2c3e50")
    cash_disc_frame.grid(row=1, column=7, columnspan=3, sticky="w", padx=(10, 2), pady=5)
    
    # ---------- Cash Discount Label ----------
    tk.Label(cash_disc_frame, text="Cash Disc.:", bg="#2c3e50", fg="white",
             font=("Segoe UI", 13, "bold")).grid(row=0, column=0, sticky="e")
    
    # ---------- Cash Discount Entry ----------
    cash_disc_var = tk.StringVar()
    entry_disc = ttk.Entry(cash_disc_frame, textvariable=cash_disc_var, width=6,font=("Segoe UI",10, "bold"))
    entry_disc.grid(row=0, column=1, padx=(5, 2), ipady=3)
    
    # ---------- % Checkbox ----------
    is_percent = tk.BooleanVar(value=True)
    percent_cb = ttk.Checkbutton(cash_disc_frame, text="%", variable=is_percent)
    percent_cb.grid(row=0, column=2, padx=(2, 2))
    
    
    def apply_discount():
        disc_val = cash_disc_var.get().strip()
        percent_mode = is_percent.get()
    
        try:
            disc_val = float(disc_val)
        except ValueError:
            tk.messagebox.showerror("Invalid Discount", "Please enter a valid discount value.")
            return
    
        total_discount = 0.0
    
        for row in all_entry_rows:
            try:
                list_price = float(row[6].get().strip())
            except:
                list_price = 0
            try:
                net_qty = float(row[5].get().strip())
            except:
                net_qty = 0
    
            base_price = list_price * net_qty
    
            if percent_mode:
                discount_amt = base_price * (disc_val / 100)
            else:
                discount_amt = disc_val
    
            # Ensure flat discount doesn't exceed base price
            if not percent_mode and discount_amt > base_price:
                discount_amt = base_price
    
            new_gross_price = base_price - discount_amt
            new_unit_price = new_gross_price / net_qty if net_qty else 0
    
            row[8].delete(0, tk.END)
            row[8].insert(0, f"{new_unit_price:.2f}")
    
            row[9].delete(0, tk.END)
            row[9].insert(0, f"{new_gross_price:.2f}")
    
            total_discount += discount_amt
    
        # Update total LP value again
        update_lp_gross_and_total()
    
        # Update summary section discount
        net_discount_var.set(f"{total_discount:.2f}")
    
    apply_btn = ttk.Button(cash_disc_frame, text="✔", width=2, command=apply_discount)
    apply_btn.grid(row=0, column=3, padx=(5, 0))


    # ---------- Row 0 ----------
    tk.Label(cust_frame, text="Phone No.:", bg="#2c3e50", fg="white",
             font=("Segoe UI", 15, "bold")).grid(row=0, column=2, sticky="e", padx=(10, 2), pady=5)
    entry_phone = ttk.Entry(cust_frame, width=22,textvariable=phone_var,font=("Segoe UI", 10, "bold"))
    entry_phone.grid(row=0, column=3, padx=(2, 15), pady=5, ipady=4)
    
    tk.Label(cust_frame, text="Bill Number:", bg="#2c3e50", fg="white",
             font=("Segoe UI", 15, "bold")).grid(row=0, column=4, sticky="e", padx=(5, 2), pady=5)
    entry_bill = ttk.Entry(cust_frame, width=22, textvariable=billno_var,font=("Segoe UI", 10, "bold"))
    entry_bill.grid(row=0, column=5, padx=(2, 15), pady=5, ipady=4)

    # ---------- Cancel Reason Label + Combobox-as-Entry ----------
    tk.Label(cust_frame, text="Remarks:", bg="#2c3e50", fg="white",
             font=("Segoe UI", 15, "bold")).grid(row=0, column=7, sticky="e", padx=(0, 0), pady=5)
    
    cancel_reasons = [
        "Customer Cancelled",
        "Out of Stock",
        "Incorrect Order",
        "Delayed Delivery",
        "Other"
    ]
    
    cancel_reason_var = tk.StringVar()
    entry_cancel_reason = ttk.Combobox(cust_frame, textvariable=cancel_reason_var, values=cancel_reasons, width=20,font=("Segoe UI", 10, "bold"))
    entry_cancel_reason.grid(row=0, column=8, padx=(0, 0), pady=5, ipady=4)
    entry_cancel_reason.set("")  # Start blank
    entry_cancel_reason.configure(state="normal")  # Editable like Entry
    
 
    
    # ---------- Row 1 ----------
    tk.Label(cust_frame, text="Shipping Address:", bg="#2c3e50", fg="white",
             font=("Segoe UI", 15, "bold")).grid(row=1, column=0, sticky="e", padx=(5, 2), pady=5)
    entry_address = ttk.Entry(cust_frame, width=45, textvariable=address_var,font=("Segoe UI", 10, "bold"))
    entry_address.grid(row=1, column=1, columnspan=2, padx=(2, 15), pady=5, ipady=4)
    
    tk.Label(cust_frame, text="Bill Date:", bg="#2c3e50", fg="white",
             font=("Segoe UI", 15, "bold")).grid(row=1, column=3, sticky="e", padx=(5, 2), pady=5)
    entry_bill_date = DateEntry(cust_frame, width=20, background='darkblue',
                                foreground='white', borderwidth=2, date_pattern='dd/mm/yyyy',font=("Segoe UI", 10, "bold"))
    entry_bill_date.set_date(datetime.today())
    entry_bill_date.grid(row=1, column=4, padx=(2, 15), pady=5, ipady=4)
    
    tk.Label(cust_frame, text="Delivery Date:", bg="#2c3e50", fg="white",
             font=("Segoe UI", 15, "bold")).grid(row=1, column=5, sticky="e", padx=(5, 2), pady=5)
    entry_delivery = DateEntry(cust_frame, width=20, background='darkblue',
                               foreground='white', borderwidth=2, date_pattern='dd/mm/yyyy',font=("Segoe UI", 10, "bold"))
    entry_delivery.set_date(datetime.today())
    entry_delivery.grid(row=1, column=6, padx=(2, 10), pady=5, ipady=4)
    
    # ---------- Initialize Globals First (before any function uses them) ----------
    current_row = 1
    popup_window = None
    all_entry_rows = []
    lp_total_var = tk.StringVar(value="0.00")
    
    # ---------- Scrollable Canvas Setup ----------
    scroll_frame = tk.Frame(root)
    scroll_frame.pack(fill="both", expand=True, padx=10, pady=5)
    
    canvas = tk.Canvas(scroll_frame, bg="white", highlightthickness=0)
    canvas.pack(side="left", fill="both", expand=True)
    
    v_scroll = tk.Scrollbar(scroll_frame, orient="vertical", command=canvas.yview)
    v_scroll.pack(side="right", fill="y")
    
    h_scroll = tk.Scrollbar(root, orient="horizontal", command=canvas.xview)
    h_scroll.pack(fill="x")
    
    canvas.configure(yscrollcommand=v_scroll.set, xscrollcommand=h_scroll.set)
    
    # ---------- Combined Frame inside Canvas (with header + table) ----------
    container_frame = tk.Frame(canvas, bg="white")  # New master frame
    canvas.create_window((0, 0), window=container_frame, anchor="nw")
    
    summary_frame1 = tk.Frame(container_frame, bg="white")  # Header inside canvas
    summary_frame1.pack(fill="x")
    
    table_frame = tk.Frame(container_frame, bg="white")
    table_frame.pack(fill="both", expand=True)
    
    # ---------- Make canvas scroll properly ----------
    def _on_mousewheel(event):
        canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")
    
    def _on_shift_mousewheel(event):
        canvas.xview_scroll(int(-1 * (event.delta / 120)), "units")
    
    container_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
    
    canvas.bind("<Enter>", lambda e: canvas.bind_all("<MouseWheel>", _on_mousewheel))
    canvas.bind("<Leave>", lambda e: canvas.unbind_all("<MouseWheel>"))
    
    canvas.bind("<Enter>", lambda e: canvas.bind_all("<Shift-MouseWheel>", _on_shift_mousewheel))
    canvas.bind("<Leave>", lambda e: canvas.unbind_all("<Shift-MouseWheel>"))

    
    cols = [
        "DESCRIPTION", "RATINGS", "CAT NO.", "MAKE",
        "QUANTITY", "NET QTY", "LIST PRICE", "DISCOUNT",
        "DISC UNIT PRICE", "DISC GROSS PRICE", "LP GRASS.PRICE"
    ]
    
    entry_font = ("Helvetica", 10)
    header_font = ("Helvetica", 10, "bold")
    header_bg = "#2e3f50"
    header_fg = "white"
    
    for j in range(len(cols)):
        summary_frame1.grid_columnconfigure(j, weight=1, uniform="col")
        table_frame.grid_columnconfigure(j, weight=1, uniform="col")
    
    for j, col in enumerate(cols):
        header = tk.Label(summary_frame1, text=col, font=header_font,
                          bg=header_bg, fg=header_fg, relief="ridge",
                          padx=2, pady=4)
        header.grid(row=0, column=j, sticky="nsew")
    
    # ---------- Utility Functions ----------
    def update_lp_gross_and_total():
        total = 0.0
        for row in all_entry_rows:
            try: qty = float(row[4].get().strip())
            except: qty = 0
            try: price = float(row[6].get().strip())
            except: price = 0
    
            lp_value = qty * price
            row[10].delete(0, tk.END)
            row[10].insert(0, f"{lp_value:.2f}")
            total += lp_value
    
        lp_total_var.set(f"{total:.2f}")
    
    def get_mode_quantity(description):
        quantities = []
        for row in all_entry_rows:
            desc = row[0].get().strip()
            qty = row[4].get().strip()
            if desc == description and qty.isdigit():
                quantities.append(qty)
        if quantities:
            count = Counter(quantities)
            mode = count.most_common(1)[0][0]
            return mode
        return ""
    
    def update_net_qty_on_change(row_index):
        row_widgets = all_entry_rows[row_index]
        description = row_widgets[0].get().strip()
        mode_qty = get_mode_quantity(description)
        if mode_qty:
            row_widgets[5].delete(0, tk.END)
            row_widgets[5].insert(0, mode_qty)

        # ---------- Sample Inventory ----------
    inventory_data = [
        {"DESCRIPTION": "MCB 10A", "RATINGS": "10A", "CAT NO.": "M10", "MAKE": "Havells",   "QUANTITY": "1", "LIST PRICE": "120.00"},
        {"DESCRIPTION": "MCB 16A", "RATINGS": "16A", "CAT NO.": "M16", "MAKE": "Siemens",   "QUANTITY": "1", "LIST PRICE": "130.00"},
        {"DESCRIPTION": "RCCB 40A", "RATINGS": "40A", "CAT NO.": "R40", "MAKE": "Anchor",    "QUANTITY": "1", "LIST PRICE": "450.00"},
        {"DESCRIPTION": "MCB 6A", "RATINGS": "6A", "CAT NO.": "M6", "MAKE": "Schneider", "QUANTITY": "1", "LIST PRICE": "110.00"},
        {"DESCRIPTION": "ELCB 63A", "RATINGS": "63A", "CAT NO.": "E63", "MAKE": "Havells",   "QUANTITY": "1", "LIST PRICE": "650.00"},
    ]

    
    def show_inventory_popup(entry_widget, row_index):
        global popup_window
    
        if popup_window:
            popup_window.destroy()
    
        typed = entry_widget.get().strip().lower()
        matches = [item for item in inventory_data if typed in item["DESCRIPTION"].lower()]
        if not matches:
            return
    
        x = entry_widget.winfo_rootx()
        y = entry_widget.winfo_rooty() + entry_widget.winfo_height()
    
        popup_window = tk.Toplevel()
        popup_window.geometry(f"700x200+{x}+{y}")
        popup_window.wm_overrideredirect(True)
    
        frame = tk.Frame(popup_window)
        frame.pack(fill="both", expand=True)
    
        tree_scroll = tk.Scrollbar(frame)
        tree_scroll.pack(side="right", fill="y")
    
        tree = ttk.Treeview(frame, columns=("DESCRIPTION", "RATINGS", "CAT NO.", "MAKE", "QUANTITY", "LIST PRICE"),
                            show="headings", height=6, yscrollcommand=tree_scroll.set)
        tree.pack(fill="both", expand=True)
        tree_scroll.config(command=tree.yview)
    
        for col in ("DESCRIPTION", "RATINGS", "CAT NO.", "MAKE", "QUANTITY", "LIST PRICE"):
            tree.heading(col, text=col)
            tree.column(col, width=100, anchor="center")
    
        for item in matches:
            values = (item["DESCRIPTION"], item["RATINGS"], item["CAT NO."], item["MAKE"], item["QUANTITY"], item["LIST PRICE"])
            tree.insert("", tk.END, values=values)
    
        def on_row_double_click(event):
            selected = tree.focus()
            if selected:
                values = tree.item(selected, "values")
                row_widgets = all_entry_rows[row_index]
                for i in range(5):
                    row_widgets[i].delete(0, tk.END)
                    row_widgets[i].insert(0, values[i])
                row_widgets[6].delete(0, tk.END)
                row_widgets[6].insert(0, values[5])
                update_lp_gross_and_total()
                update_net_qty_on_change(row_index)
            popup_window.destroy()
    
        tree.bind("<Double-1>", on_row_double_click)

    def delete_row(row_index):
        if row_index >= len(all_entry_rows):
            print(f"Cannot delete row at index {row_index}: out of range.")
            return
    
        for widget in all_entry_rows[row_index]:
            widget.destroy()
    
        del all_entry_rows[row_index]
        
        update_lp_gross_and_total()
    
    def on_discount_backspace(event, idx):
        if 0 <= idx < len(all_entry_rows):
            delete_row(idx)
        else:
            print(f"Invalid index {idx} for deletion.")

        
    # ---------- Entry Row ----------
    def insert_entry_row(data=[]):
        global current_row, all_entry_rows
        row_widgets = []
        row_index = len(all_entry_rows)
    
        for j in range(len(cols)):
            value = data[j] if j < len(data) else ""
            e = tk.Entry(table_frame, font=entry_font, justify="center", bg="white", relief="solid", borderwidth=1)
            e.grid(row=current_row, column=j, sticky="nsew", padx=1, pady=2, ipady=3)

            e.insert(0, value)
            row_widgets.append(e)
    
            # NOTE: This is now handled globally at the start
            # table_frame.grid_columnconfigure(j, weight=1)
    
            if cols[j] == "DESCRIPTION":
                e.bind("<KeyRelease>", lambda event, ent=e, idx=row_index: show_inventory_popup(ent, idx))
    
            if cols[j] == "QUANTITY":
                def on_qty_change(event, idx=row_index):
                    update_net_qty_on_change(idx)
                    update_lp_gross_and_total()
                e.bind("<KeyRelease>", on_qty_change)
    
            if cols[j] == "LIST PRICE":
                e.bind("<KeyRelease>", lambda event: update_lp_gross_and_total())
    
            if cols[j] == "DISCOUNT":
                def on_discount_tab(event, row_idx=row_index):
                    entry = event.widget
                    discount_str = entry.get().strip().replace('%', '')
                    try:
                        discount = float(discount_str)
                    except:
                        discount = 0
                    entry.delete(0, tk.END)
                    entry.insert(0, f"{discount}%")
    
                    row_widgets = all_entry_rows[row_idx]
                    try:
                        list_price = float(row_widgets[6].get().strip())
                    except:
                        list_price = 0
                    try:
                        net_qty = float(row_widgets[5].get().strip())
                    except:
                        net_qty = 0
    
                    disc_unit_price = list_price - (list_price * discount / 100)
                    disc_gross_price = disc_unit_price * net_qty
    
                    row_widgets[8].delete(0, tk.END)
                    row_widgets[8].insert(0, f"{disc_unit_price:.2f}")
                    row_widgets[9].delete(0, tk.END)
                    row_widgets[9].insert(0, f"{disc_gross_price:.2f}")
    
                    update_lp_gross_and_total()
                    insert_entry_row()
                    return "break"
    
                def on_discount_return(event):
                    event.widget.tk_focusNext().focus()
                    insert_section_heading_input()
                    return "break"
    
                e.bind("<Tab>", on_discount_tab)
                e.bind("<Return>", on_discount_return)
                e.bind("<KeyRelease-BackSpace>", lambda event, i=row_index: on_discount_backspace(event, i))
    
        all_entry_rows.append(row_widgets)
        current_row += 1

   
    # ---------- Section Heading ----------
    def insert_section_heading_input():
        global current_row
        entry = tk.Entry(table_frame, font=("Helvetica", 11, "bold"), bg="#d0e0f0", justify="left")
        entry.grid(row=current_row, column=0, columnspan=len(cols), sticky="nsew", pady=(8, 4), ipady=3)
        entry.focus()
        entry.bind("<Return>", process_section_heading)
    
    def process_section_heading(event):
        global current_row
        heading_text = event.widget.get().strip()
        if heading_text:
            event.widget.grid_forget()
            label = tk.Label(table_frame, text=heading_text, font=("Helvetica", 11, "bold"),
                             bg="#d0e0f0", anchor="w", padx=10, pady=6, relief="ridge")
            label.grid(row=current_row, column=0, columnspan=len(cols), sticky="nsew", pady=(8, 4))
            current_row += 1
            insert_entry_row()
    
    # ---------- Start ----------
    insert_section_heading_input()
    # ----------------- Bill Summary Frame -----------------
    summary_frame = tk.LabelFrame(root, text="Bill Menu", font=("Segoe UI", 11, "bold"),
                                  bg="#2c3e50", fg="white", padx=10, pady=10)
    summary_frame.pack(fill="x", padx=10, pady=5)


    enclosure_total_var = tk.StringVar(value="₹ 0.00")
    
    def open_enclosure(total_var=None):
        popup = tk.Toplevel()
        popup.title("Enclosure Fabrication")
        popup.geometry("1100x700")
        popup.configure(bg="white")
    
        headers = ["✔", " ", "MM", "Length", "Width", "", "Kg", "", "Kg per Sqmm"]
    
        enclosure_data = [
            {" ": "18 SWG", "MM": 1.20, "Length": 2500, "Width": 1250, "Kg": 30, "": 29.4375, "Kg per Sqmm": 0.0000096},
            {" ": "16 SWG", "MM": 1.50, "Length": 2500, "Width": 1250, "Kg": 40, "": 36.796875, "Kg per Sqmm": 0.0000128},
            {" ": "14 SWG", "MM": 2.00, "Length": 2500, "Width": 1250, "Kg": 50, "": 49.0625, "Kg per Sqmm": 0.000016},
            {" ": "13 SWG", "MM": 2.30, "Length": 2500, "Width": 1250, "Kg": 60, "": 56.421875, "Kg per Sqmm": 0.0000192},
            {" ": "12 SWG", "MM": 2.64, "Length": 2500, "Width": 1250, "Kg": 70, "": 64.7625, "Kg per Sqmm": 0.0000224},
            {" ": "11 SWG", "MM": 3.00, "Length": 2500, "Width": 1250, "Kg": 80, "": 73.59375, "Kg per Sqmm": 0.0000256},
            {" ": "10 SWG", "MM": 3.25, "Length": 2500, "Width": 1250, "Kg": 90, "": 79.7265625, "Kg per Sqmm": 0.0000288},
        ]
    
        entry_matrix1 = []
        checkbox_vars = []
    
        def format_decimal(value):
            return "{:.10f}".format(value).rstrip("0").rstrip(".") if "." in "{:.10f}".format(value) else "{:.10f}".format(value)
        
        def update_total(row_idx):
            try:
                mm = float(entry_matrix1[row_idx][1].get())
                length = float(entry_matrix1[row_idx][2].get())
                width_val = float(entry_matrix1[row_idx][3].get())
                total = round(mm * length * width_val * 0.00000785, 2)
                entry_matrix1[row_idx][4].config(state='normal')
                entry_matrix1[row_idx][4].delete(0, tk.END)
                entry_matrix1[row_idx][4].insert(0, total)
                entry_matrix1[row_idx][4].config(state='readonly', disabledforeground="black")
            except ValueError:
                entry_matrix1[row_idx][4].config(state='normal')
                entry_matrix1[row_idx][4].delete(0, tk.END)
                entry_matrix1[row_idx][4].insert(0, "Error")
                entry_matrix1[row_idx][4].config(state='readonly', disabledforeground="red")
        
        def update_kg_per_sqmm(row_idx):
            try:
                kg = float(entry_matrix1[row_idx][5].get())
                length = float(entry_matrix1[row_idx][2].get())
                width_val = float(entry_matrix1[row_idx][3].get())
                if length * width_val == 0:
                    formatted_result = "0.0000000000"
                else:
                    result = kg / (length * width_val)
                    formatted_result = format_decimal(result)
                entry_matrix1[row_idx][8].config(state='normal')
                entry_matrix1[row_idx][8].delete(0, tk.END)
                entry_matrix1[row_idx][8].insert(0, formatted_result)
                entry_matrix1[row_idx][8].config(state='readonly', disabledforeground="black")
            except ValueError:
                entry_matrix1[row_idx][8].config(state='normal')
                entry_matrix1[row_idx][8].delete(0, tk.END)
                entry_matrix1[row_idx][8].insert(0, "Error")
                entry_matrix1[row_idx][8].config(state='readonly', disabledforeground="red")
        
        popup.grid_columnconfigure(0, minsize=30)
        
        # Header labels for first table
        for col, text in enumerate(headers):
            bg_color = "#A1AEB1"
            fg_color = "black"
            col_width = 14  # <-- renamed from `width` to avoid name conflict
            lbl = tk.Label(popup, text=text if text else "Calculated",
                           font=('times new roman', 12, "bold"),
                           bg=bg_color, fg=fg_color, width=col_width, borderwidth=1, relief="solid")
            lbl.grid(row=0, column=col, padx=1, pady=8, sticky="nsew")
        
        # Populate first table rows
        for row_index, data in enumerate(enclosure_data):
            row_entries = []
            var = tk.IntVar()
            checkbox_vars.append(var)
        
            # Checkbox column
            chk = tk.Checkbutton(popup, variable=var, bg="white", padx=0, pady=0, takefocus=0)
            chk.grid(row=row_index + 1, column=0, padx=1, pady=3, sticky="w")
            row_entries.append(None)
        
            for col_index, header in enumerate(headers[1:], start=1):
                ent = tk.Entry(popup, font=('times new roman', 12), width=14, borderwidth=1, relief="solid", justify='center')
                ent.grid(row=row_index + 1, column=col_index, padx=1, pady=3, sticky="nsew")
        
                # Insert initial values or calculated fields
                if header in data:
                    ent.insert(0, data[header])
                elif col_index == 4:  # MM*L*W*0.00000785
                    mm = float(data.get("MM", 0))
                    length = float(data.get("Length", 0))
                    width_val = float(data.get("Width", 0))
                    total = round(mm * length * width_val * 0.00000785, 2)
                    ent.insert(0, total)
                    ent.config(state='readonly', disabledforeground="black")
                elif col_index == 8:  # Kg / (L * W)
                    kg = float(data.get("Kg", 0))
                    length = float(data.get("Length", 0))
                    width_val = float(data.get("Width", 0))
                    if length * width_val > 0:
                        result = kg / (length * width_val)
                        formatted_result = format_decimal(result)
                    else:
                        formatted_result = "0.0000000000"
                    ent.insert(0, formatted_result)
                    ent.config(state='readonly', disabledforeground="black")
        
                # Bind events for live calculation
                if col_index in [1, 2, 3]:  # MM, Length, Width affect total
                    ent.bind("<FocusOut>", lambda e, r=row_index: update_total(r))
                    ent.bind("<Return>", lambda e, r=row_index: update_total(r))
                    ent.bind("<Tab>", lambda e, r=row_index: update_total(r))
                if col_index in [2, 3, 5]:  # Length, Width, Kg affect KG/sqmm
                    ent.bind("<FocusOut>", lambda e, r=row_index: update_kg_per_sqmm(r))
                    ent.bind("<Return>", lambda e, r=row_index: update_kg_per_sqmm(r))
                    ent.bind("<Tab>", lambda e, r=row_index: update_kg_per_sqmm(r))
        
                # Optional row highlight
                if row_index in [0, 2, 4, 6]:
                    ent.config(bg="#FF7F7F")
        
                row_entries.append(ent)
            entry_matrix1.append(row_entries)

    
        # Second table headers and initial data
        headers1 = ["", "L", "W", "H", "Volume"]
        second_table_row_start = len(enclosure_data) + 3
        col_width = 14
    
        header_bg = "#FFD700"
        header_fg = "#4B4B4B"
    
        Enclosure_data1 = [
            {"": "28+1+1", "L": 2400, "W": 300, "H": 1600, "Volume": ""},
        ]
    
        autofill_map = {
            1: [1, 3],  # L column autofill rows
            2: [3],     # W column autofill rows
        }
    
        fallback_values = {
            (2, 1): 300,
            (1, 2): 1600,
            (2, 2): 1600,
        }
    
        entry_matrix2 = []
    
        def smart_autofill(col_index, value):
            if col_index in [3, 4]:  # Skip autofill for H and Volume
                return
    
            filled_rows = set()
            for row in autofill_map.get(col_index, []):
                entry_matrix2[row][col_index].delete(0, tk.END)
                entry_matrix2[row][col_index].insert(0, value)
                filled_rows.add(row)
    
            for row in range(1, 5):
                if (row, col_index) not in filled_rows:
                    cell = entry_matrix2[row][col_index]
                    if not cell.get().strip():
                        fallback = fallback_values.get((row, col_index))
                        if fallback is not None:
                            cell.insert(0, fallback)
    
        # Second table header row
        for col, text in enumerate(headers1):
            lbl = tk.Label(popup, text=text, font=('times new roman', 12, "bold"),
                           bg=header_bg, fg=header_fg, width=col_width, borderwidth=1, relief="solid")
            lbl.grid(row=second_table_row_start, column=col, padx=1, pady=(15, 0), sticky="nsew")
    
        # Second table entry grid
        for row in range(6):
            row_entries = []
            for col, header in enumerate(headers1):
                bg_color = header_bg if row == 0 else "white"
                ent = tk.Entry(popup, font=('times new roman', 12), bg=bg_color, width=col_width,
                               borderwidth=1, relief="solid", justify='center')
                ent.grid(row=second_table_row_start + row + 1, column=col, padx=1, pady=3, sticky="nsew")
    
                if row == 0:
                    ent.insert(0, Enclosure_data1[0].get(header, ""))
                    def make_callback(c=col):
                        def on_focus_out(event):
                            smart_autofill(c, event.widget.get())
                        return on_focus_out
                    ent.bind("<FocusOut>", make_callback())
                elif col == 0 and 1 <= row <= 5:
                    ent.insert(0, str(row))  # serial numbers 1-5
    
                row_entries.append(ent)
            entry_matrix2.append(row_entries)
    
        # Total calculation function
        def enclosure_calculate_total():
            total_volume = 0
            for row in entry_matrix2[1:7]:
                try:
                    l = float(row[1].get())
                    w = float(row[2].get())
                    h = float(row[3].get())
                    volume = l * w * h
                    total_volume += volume
                    row[4].config(state='normal')
                    row[4].delete(0, tk.END)
                    row[4].insert(0, f"{int(volume)}")
                    row[4].config(state='readonly', disabledforeground="black")
                except ValueError:
                    row[4].config(state='normal')
                    row[4].delete(0, tk.END)
                    row[4].config(state='readonly')
            enclosure_total_var.set(str(int(total_volume)))
    
        btn_row = second_table_row_start + 7
    
        # Percent section (left aligned)
        percent_label = tk.Label(popup, text="Percent:", font=('times new roman', 12, "bold"), bg="white")
        percent_label.grid(row=btn_row - 1, column=0, pady=8, padx=(15,5), sticky="w")
    
        percent_entry = tk.Entry(popup, font=('times new roman', 12), width=18)
        percent_entry.grid(row=btn_row - 1, column=1, pady=8, padx=5, sticky="w")
    
        percent_result_entry = tk.Entry(popup, font=('times new roman', 12), width=18, state='readonly',
                                       disabledforeground="black")
        percent_result_entry.grid(row=btn_row - 1, column=2, pady=8, padx=5, sticky="w")
    
        percent_final_entry = tk.Entry(popup, font=('times new roman', 12), width=18, state='readonly',
                                      disabledforeground="black")
        percent_final_entry.grid(row=btn_row - 1, column=3, pady=8, padx=5, sticky="w")
    
        # Buttons aligned nicely below
        tk.Button(popup, text="Total", command=enclosure_calculate_total,
                  font=('times new roman', 12, "bold"), bg="#0f4d7d", fg="white")\
            .grid(row=btn_row, column=1, pady=12, padx=15, sticky="w")
    
        tk.Button(popup, text="Close", command=popup.destroy,
                  font=('times new roman', 12, "bold"), bg="#0f4d7d", fg="white")\
            .grid(row=btn_row, column=0, pady=12, padx=15, sticky="e")
    
        def apply_percent_to_total(event=None):
            try:
                enclosure_calculate_total()
                percent_text = percent_entry.get().strip()
                if not percent_text:
                    percent_result_entry.config(state='normal')
                    percent_result_entry.delete(0, tk.END)
                    percent_result_entry.config(state='readonly')
                    percent_final_entry.config(state='normal')
                    percent_final_entry.delete(0, tk.END)
                    percent_final_entry.config(state='readonly')
                    return
    
                if not percent_text.endswith("%"):
                    percent_text += "%"
    
                percent_entry.delete(0, tk.END)
                percent_entry.insert(0, percent_text)
    
                percent = float(percent_text.rstrip("%"))
                total_volume = float(enclosure_total_var.get())
    
                result = (percent / 100) * total_volume
                final = total_volume + result
    
                percent_result_entry.config(state='normal')
                percent_result_entry.delete(0, tk.END)
                percent_result_entry.insert(0, f"{round(result, 2)}")
                percent_result_entry.config(state='readonly')
    
                percent_final_entry.config(state='normal')
                percent_final_entry.delete(0, tk.END)
                percent_final_entry.insert(0, f"{round(final, 2)}")
                percent_final_entry.config(state='readonly')
    
            except Exception:
                percent_result_entry.config(state='normal')
                percent_result_entry.delete(0, tk.END)
                percent_result_entry.config(state='readonly')
    
                percent_final_entry.config(state='normal')
                percent_final_entry.delete(0, tk.END)
                percent_final_entry.config(state='readonly')
    
        percent_entry.bind("<FocusOut>", apply_percent_to_total)
        percent_entry.bind("<Return>", apply_percent_to_total)
    
        # Initial total calculation
        enclosure_calculate_total()

    

        



    wiring_total_var = tk.StringVar(value="₹ 0.00")
    
    def open_wiring(total_var=None):
        popup = tk.Toplevel()
        popup.title("Wiring + Accessories")
        popup.geometry("850x460")  # Fits everything without scroll
        popup.configure(bg="#eef2f3")
        popup.resizable(False, False)
    
        headers = ["SIZE", "RATE", "QTY", "TOTAL"]
    
        predefined_items = [
            (35, 350),
            (25, 250),
            (16, 170),
            (10, 110),
            (6, 60),
            (4, 40),
            (2.5, 30),
            (1, 15)
        ]
    
        # Main frame without scrollbars
        frame = tk.Frame(popup, bg="#eef2f3")
        frame.pack(padx=15, pady=15)
    
        # Table Header
        for col, text in enumerate(headers):
            lbl = tk.Label(
                frame,
                text=text,
                font=('Calibri', 12, "bold"),
                bg="#0f4d7d",
                fg="white",
                width=20,
                pady=8,
                bd=1,
                relief="solid"
            )
            lbl.grid(row=0, column=col, padx=1, pady=1)
    
        entry_matrix = []
    
        def wiring_calculate_total():
            total = 0
            for row in entry_matrix:
                try:
                    qty = float(row[2].get())
                    rate = float(row[1].get())
                    row_total = qty * rate
                    row[3].config(state="normal")
                    row[3].delete(0, tk.END)
                    row[3].insert(0, f"{row_total:.2f}")
                    row[3].config(state="readonly")
                    total += row_total
                except:
                    row[3].config(state="normal")
                    row[3].delete(0, tk.END)
                    row[3].insert(0, "")
                    row[3].config(state="readonly")
                    continue
            wiring_total_var.set(f"₹ {total:.2f}")
    
        def wiring_add_row(size="", rate=""):
            row_idx = len(entry_matrix) + 1
            entries = []
            bg_color = "#ffffff" if row_idx % 2 == 0 else "#f1f9ff"
    
            e1 = tk.Entry(frame, width=20, bg=bg_color, font=('Calibri', 11), relief="solid", bd=1)
            e1.insert(0, size)
            e1.grid(row=row_idx, column=0, padx=1, pady=2)
            entries.append(e1)
    
            e2 = tk.Entry(frame, width=20, bg=bg_color, font=('Calibri', 11), relief="solid", bd=1)
            e2.insert(0, rate)
            e2.grid(row=row_idx, column=1, padx=1, pady=2)
            entries.append(e2)
    
            e3 = tk.Entry(frame, width=20, bg=bg_color, font=('Calibri', 11), relief="solid", bd=1)
            e3.grid(row=row_idx, column=2, padx=1, pady=2)
            entries.append(e3)
    
            e4 = tk.Entry(frame, width=20, state="readonly", readonlybackground="#e0f0ff",
                          font=('Calibri', 11), relief="solid", bd=1)
            e4.grid(row=row_idx, column=3, padx=1, pady=2)
            entries.append(e4)
    
            def on_enter(event):
                wiring_calculate_total()
                if entries == entry_matrix[-1]:
                    wiring_add_row()
    
            e3.bind("<Return>", on_enter)
            entry_matrix.append(entries)
    
        for size, rate in predefined_items:
            wiring_add_row(size, rate)
    
        # Bottom Buttons
        btn_frame = tk.Frame(popup, bg="#eef2f3")
        btn_frame.pack(pady=15)
    
        button_style = {
            "font": ('Calibri', 12, "bold"),
            "bg": "#0f4d7d",
            "fg": "white",
            "activebackground": "#155a8a",
            "activeforeground": "white",
            "bd": 0,
            "width": 14,
            "cursor": "hand2"
        }
    
        tk.Button(btn_frame, text="Calculate Total", command=wiring_calculate_total, **button_style)\
            .grid(row=0, column=0, padx=10)
    
        tk.Button(btn_frame, text="Close", command=popup.destroy, **button_style)\
            .grid(row=0, column=1, padx=10)
        wiring_total_var = tk.StringVar(value="₹ 0.00")

        entry_wiring = ttk.Entry(summary_frame, textvariable=wiring_total_var, width=15)
        entry_wiring.grid(row=0, column=3, padx=5, pady=5)


    section_start_index = {}
    busbar_sections = {}
    def open_busbar():
        global table_frame, all_entries, busbar_total_label
    
        popup = tk.Toplevel()
        popup.title("BUS BAR")
        popup.geometry("1400x750")
        popup.configure(bg="white")
    
        canvas_frame = tk.Frame(popup, bg="white")
        canvas_frame.pack(fill=tk.BOTH, expand=True)
    
        canvas = tk.Canvas(canvas_frame, bg="white", highlightthickness=0)
        canvas.grid(row=0, column=0, sticky="nsew")
    
        scrollbar_x = ttk.Scrollbar(canvas_frame, orient="horizontal", command=canvas.xview)
        scrollbar_x.grid(row=1, column=0, sticky="ew")
        canvas.configure(xscrollcommand=scrollbar_x.set)
    
        canvas_frame.grid_rowconfigure(0, weight=1)
        canvas_frame.grid_columnconfigure(0, weight=1)
    
        scrollable_frame = tk.Frame(canvas, bg="white")
        window_id = canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    
        def on_frame_configure(event):
            canvas.configure(scrollregion=canvas.bbox("all"))
            canvas.itemconfig(window_id, width=max(scrollable_frame.winfo_reqwidth(), canvas.winfo_width()))
    
        scrollable_frame.bind("<Configure>", on_frame_configure)
    
        def _on_shiftmousewheel(event):
            canvas.xview_scroll(int(-1 * (event.delta / 120)), "units")
    
        canvas.bind_all("<Shift-MouseWheel>", _on_shiftmousewheel)
    
        headers = [
            "L", "W", "RUN", "WEIGHT (KG/M)", "PHASE", "NEUTRAL",
            "PANEL LENGTH", "NO. OF HORI", "TOTAL LENGTH (Mts)",
            "WEIGHT OF THE BUS BAR", "BUS BAR PRICE", "GRAND PRICE"
        ]
        num_cols = len(headers)
        current_row = 0
        all_entries = []
        material_density = tk.DoubleVar(value=0.0)
    
        style = ttk.Style()
        style.theme_use('clam')
        style.configure("CustomCombobox.TCombobox",
                        fieldbackground="white",
                        background="#0f4d7d",
                        foreground="black",
                        font=("Arial", 10),
                        padding=5)
    
        top_frame = tk.Frame(scrollable_frame, bg="#e6f2ff")
        top_frame.grid(row=current_row, column=0, sticky="ew", padx=10, pady=(5, 2))
        tk.Label(top_frame, text="BUS BAR", font=("Arial", 10, "bold"),
                 bg="#e6f2ff", fg="#003366").grid(row=0, column=1)
    
        selected_busbar = tk.StringVar()
        busbar_options = [
            "VERTICAL-INCOMER FOR B/C", "VERTICAL-O/G FOR Ph1", "VERTICAL-O/G FOR Ph2",
            "VERTICAL-APFC O/G", "VERTICAL-INCOMER", "VERTICAL-OUTGOING",
            "CONTROL BUS", "EARTH"
        ]
        
        combobox = ttk.Combobox(top_frame, textvariable=selected_busbar,
                                values=busbar_options,
                                state="readonly", width=30, style="CustomCombobox.TCombobox")
        combobox.set("Select Busbar Type")
        combobox.grid(row=0, column=1, sticky="w", padx=(20, 5))
        
        # New Entry + Add Button for user-defined busbars
        new_busbar_name = tk.StringVar()
        
        new_entry = tk.Entry(top_frame, textvariable=new_busbar_name, font=("Arial", 10), width=30)
        new_entry.grid(row=0, column=2, padx=5)
        
        def add_custom_busbar():
            name = new_busbar_name.get().strip()
            if not name:
                return
            if name in busbar_options:
                return  # Already exists
        
            busbar_options.append(name)
            combobox.config(values=busbar_options)
            create_section(name, [["" for _ in range(num_cols)]])
            new_busbar_name.set("")
        
        add_button = tk.Button(top_frame, text="Add", command=add_custom_busbar,
                               bg="#17a2b8", fg="white", font=("Arial", 9, "bold"))
        add_button.grid(row=0, column=3, padx=(5, 10))
        current_row += 1
    
        sub_frame = tk.Frame(scrollable_frame, bg="#d9fdd3")
        sub_frame.grid(row=current_row, column=0, sticky="ew", padx=10)
    
        def on_material_select(event):
            selection = sub_combobox.get()
            if "Aluminum" in selection:
                material_density.set(0.0027)
            elif "Copper" in selection:
                material_density.set(0.0089)
            else:
                material_density.set(0.0)
    
        sub_combobox = ttk.Combobox(sub_frame, state="readonly",
                                     values=["Aluminum = 0.0027", "Copper = 0.0089"],
                                     width=20, font=("Arial", 10),
                                     style="CustomCombobox.TCombobox")
        sub_combobox.set(" ")
        sub_combobox.bind("<<ComboboxSelected>>", on_material_select)
        sub_combobox.grid(row=0, column=0, sticky="w", padx=10)
    
        tk.Label(sub_frame, text="HORIZONTAL - MAIN TRFO & DG1",
                 font=("Arial", 10, "bold"), bg="#d9fdd3", fg="#004d00") \
            .grid(row=0, column=1, sticky="w", padx=10)
        current_row += 1
    
        busbar_data = {
            "VERTICAL-INCOMER FOR B/C": [["25", "10", "1", "", "3", "", "1600", "1", "", "", "400", ""]],
            "VERTICAL-O/G FOR Ph1": [["30", "5", "2", "", "1", "1", "1800", "2", "", "", "500", ""]],
            "VERTICAL-O/G FOR Ph2": [["40", "5", "2", "", "2", "1", "1500", "1", "", "", "520", ""]],
            "VERTICAL-APFC O/G": [["50", "10", "2", "", "1", "1", "1800", "1", "", "", "600", ""]],
            "VERTICAL-INCOMER": [["20", "10", "1", "", "1", "0", "1600", "1", "", "", "450", ""]],
            "VERTICAL-OUTGOING": [["20", "10", "1", "", "0", "1", "1600", "1", "", "", "450", ""]],
            "CONTROL BUS": [["10", "5", "1", "", "1", "1", "1600", "2", "", "", "300", ""]],
            "EARTH": [["16", "5", "1", "", "0", "1", "1600", "1", "", "", "200", ""]]
        }
    
        def on_last_cell_tab(event):
            widget = event.widget
            if all_entries and widget == all_entries[-1][-1]:
                add_new_row()
                return "break"
    
        def add_new_row():
            row_index = len(all_entries) + 1
            row_entries = []
            for c in range(num_cols):
                e = tk.Entry(table_frame, font=("Arial", 10), relief="solid", justify="center")
                e.grid(row=row_index, column=c, padx=1, pady=1, sticky="nsew")
                if c == num_cols - 1:
                    e.bind("<Tab>", on_last_cell_tab)
                row_entries.append(e)
            all_entries.append(row_entries)
    
        def create_section(title, data, deletable=False):
            nonlocal current_row
            global table_frame

            section_row = current_row

            section_title_frame = tk.Frame(scrollable_frame, bg="#728FCE")
            section_title_frame.grid(row=current_row, column=0, sticky="ew", padx=10, pady=(10, 2))

            section_label = tk.Label(section_title_frame, text=title, font=("Arial", 10, "bold"),
                                     bg="#728FCE", anchor="w")
            section_label.pack(side=tk.LEFT, fill=tk.X, expand=True)

            if deletable:
                def delete_section():
                    # Destroy the widgets
                    section_title_frame.destroy()
                    table_frame.destroy()

                    # Remove entries from all_entries
                    start_idx = section_start_index.get(title, 0)
                    num_rows = len(data)
                    del all_entries[start_idx:start_idx + num_rows]

                    # Remove from tracking dicts
                    if title in busbar_sections:
                        del busbar_sections[title]
                    if title in section_start_index:
                        del section_start_index[title]

                delete_button = tk.Button(F3, text="Delete Table", command=delete_table,
                          font=("times new roman", 12, "bold"),
                          bg="red", fg="white", padx=10, pady=5)
                delete_button.grid(row=0, column=2, padx=10)

            current_row += 1

            table_frame = tk.Frame(scrollable_frame, bg="white")
            table_frame.grid(row=current_row, column=0, sticky="ew", padx=10)

            for i in range(num_cols):
                table_frame.grid_columnconfigure(i, weight=1)

            for col, header in enumerate(headers):
                lbl = tk.Label(table_frame, text=header, font=("Arial", 9, "bold"),
                               bg="#ffe0b3", relief="solid", bd=1, width=14, anchor="center")
                lbl.grid(row=0, column=col, sticky="nsew", padx=1, pady=1)

            section_start_index[title] = len(all_entries)

            for r_idx, row_data in enumerate(data, start=1):
                row_entries = []
                for c in range(num_cols):
                    e = tk.Entry(table_frame, font=("Arial", 10), relief="solid", justify="center")
                    e.grid(row=r_idx, column=c, padx=1, pady=1, sticky="nsew")
                    if c < len(row_data):
                        e.insert(0, row_data[c])
                    if c == num_cols - 1:
                        e.bind("<Tab>", on_last_cell_tab)
                    row_entries.append(e)
                all_entries.append(row_entries)

            busbar_sections[title] = (section_title_frame, table_frame)
            current_row += 1
    
        def calc_all():
            density = material_density.get()
            for row in all_entries:
                try:
                    l = float(row[0].get() or 0)
                    w = float(row[1].get() or 0)
                    run = float(row[2].get() or 0)
                    weight_kgm = l * w * run * density
                    row[3].delete(0, tk.END)
                    row[3].insert(0, f"{weight_kgm:.4f}")
    
                    phase = int(row[4].get() or 0)
                    neutral = int(row[5].get() or 0)
                    panel_len = float(row[6].get() or 0)
                    no_of_hori = float(row[7].get() or 0)
                    total_len = (panel_len / 1000) * no_of_hori * (phase + neutral)
                    row[8].delete(0, tk.END)
                    row[8].insert(0, f"{total_len:.2f}")
    
                    weight_busbar = total_len * weight_kgm
                    row[9].delete(0, tk.END)
                    row[9].insert(0, f"{weight_busbar:.2f}")
    
                    price = float(row[10].get() or 0)
                    grand = weight_busbar * price
                    row[11].delete(0, tk.END)
                    row[11].insert(0, f"{grand:,.2f}")
                except Exception as e:
                    print("Row Error:", e)
    
        def show_grand_total():
            total = 0.0
            for row in all_entries:
                try:
                    val = float(row[11].get().replace(",", "") or 0)
                    total += val
                except:
                    pass
            busbar_total_var.set(f"{total:,.2f}")

    
        def on_combobox_select(event):
            selection = selected_busbar.get()
            if selection in busbar_data:
                create_section(selection, busbar_data[selection])


    
        combobox.bind("<<ComboboxSelected>>", on_combobox_select)
        busbar_total_var = tk.StringVar(value="0.00")  # Create a StringVar
    
        bottom_button_frame = tk.Frame(popup, bg="white")
        bottom_button_frame.pack(fill=tk.X, side=tk.BOTTOM, anchor="e")
    
        calculate_btn = tk.Button(bottom_button_frame, text="Calculate All", command=calc_all,
                                   bg="#007bff", fg="white", font=("Arial", 10, "bold"))
        calculate_btn.pack(side=tk.RIGHT, padx=10, pady=5)
    
        total_btn = tk.Button(bottom_button_frame, text="Total", command=show_grand_total,
                              bg="#28a745", fg="white", font=("Arial", 10, "bold"))
        total_btn.pack(side=tk.RIGHT, padx=10, pady=5)

        
  


    # ---------- Summary Frame (Buttons + Entry Fields) ----------
    summary_frame = tk.Frame(root, bg="#2e3f50")
    summary_frame.pack(fill="x", padx=10, pady=(5, 10))
    
    buttons = [
        ("ENCLOSURE FABRICATION", open_enclosure),
        ("WIRING + ACCESSORIES", open_wiring),
        ("BUS BAR", open_busbar),
        ("TRANSPORTATION", None),
        ("LABOUR", None)
    ]
    
    entry_vars = []
    
    for i, (text, cmd) in enumerate(buttons):
        # Create Button
        btn = ttk.Button(summary_frame, text=text, width=22, command=cmd if cmd else lambda: None)
        btn.grid(row=0, column=i*2, padx=5, pady=5, sticky="ew")
    
        # Create associated Entry
        entry = ttk.Entry(summary_frame, width=22)

        entry.grid(row=0, column=i*2 + 1, padx=5, pady=5)
        entry_vars.append(entry)
    
    # Optional: expand button columns
    for i in range(len(buttons) * 2):
        summary_frame.grid_columnconfigure(i, weight=1)
        
        # --- LP GROSS TOTAL Label ---
        tk.Label(summary_frame, text="LP GROSS TOTAL:", font=("Segoe UI", 12, "bold"),
                 bg="#2c3e50", fg="white").grid(row=1, column=0, sticky="e", padx=10, pady=5)
        
        tk.Label(summary_frame, textvariable=lp_total_var, font=("Segoe UI", 12,"bold"),
                 bg="white", fg="black", width=15, anchor="w", relief="solid").grid(row=1, column=1, sticky="w", padx=5, pady=5)
        
        # --- NET DISCOUNT Label ---
        tk.Label(summary_frame, text="NET DISC:", font=("Segoe UI", 12, "bold"),
                 bg="#2c3e50", fg="white").grid(row=1, column=2, sticky="e", padx=10, pady=5)
        
        net_discount_var = tk.StringVar(value="0.00")  # Declare this before if not already
        
        tk.Label(summary_frame, textvariable=net_discount_var, font=("Segoe UI", 12,"bold"),
                 bg="white", fg="black", width=15, anchor="w", relief="solid").grid(row=1, column=3, sticky="w", padx=5, pady=5)
          
        generate_btn = ttk.Button(summary_frame, text="GENERATE BILL", width=20)
        generate_btn.grid(row=1, column=8, padx=10)

        cancel_btn = ttk.Button(summary_frame, text="CANCEL", width=20)
        cancel_btn.grid(row=1, column=9, padx=10)


    root.mainloop()

Billing_Entry_UI()
 
 
 

In [117]:
## update code by pratiksha



 

import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from datetime import datetime
from tkinter import *
from tkcalendar import DateEntry
from datetime import datetime
from collections import Counter
import sqlite3
import os


def Billing_Entry_UI():
    global current_row
    global all_entry_rows
    global popup_window
    global lp_gross_price_var
    current_row=0
    root = tk.Tk()
    root.title("Billing Management System")
    root.geometry("1400x750")
    root.configure(bg="#ecf0f1")

    # ----------------- Styling -----------------
    style = ttk.Style()
    style.theme_use("clam")

    style.configure("TLabel", font=("Segoe UI", 10), background="#2c3e50", foreground="white")
    style.configure("Header.TLabel", font=("Segoe UI", 20, "bold"), background="#2c3e50", foreground="white")
    style.configure("TButton", font=("Segoe UI", 10), background="#3498db", foreground="white")
    style.configure("TEntry", font=("Segoe UI", 10))
    style.configure("Treeview.Heading", font=("Segoe UI", 10, "bold"), background="#34495e", foreground="white")
    style.configure("Treeview", font=("Segoe UI", 10), rowheight=30)

    # ----------------- Top Header -----------------
    top_frame = tk.Frame(root, bg="#2c3e50", height=60)
    top_frame.pack(side="top", fill="x")

    back_btn = ttk.Button(top_frame, text="← Back")
    back_btn.place(x=10, y=15)

    title_lbl = ttk.Label(top_frame, text="Billing Entry", style="Header.TLabel")
    title_lbl.pack(pady=10)

    # ----------------- Customer Details -----------------
    name_var = tk.StringVar()
    phone_var = tk.StringVar()
    billno_var = tk.StringVar()
    address_var = tk.StringVar()

    cust_frame = tk.LabelFrame(root, text="Customer Details", font=("Segoe UI", 15, "bold"),
                               bg="#2c3e50", fg="white", padx=10, pady=10)
    cust_frame.pack(fill="x", padx=10, pady=5)

    # Labels & Entry Fields
    tk.Label(cust_frame, text="Customer Name:", bg="#2c3e50", fg="white", font=("Segoe UI", 15,"bold")).grid(row=0, column=0, sticky="w", padx=5, pady=5)
    entry_name = ttk.Entry(cust_frame, width=30)
    entry_name.grid(row=0, column=1, padx=5)

    # Your existing fetch_customer_details with minor adjustment
    def fetch_customer_details():
        billno = billno_var.get().strip()
        customer_name = name_var.get().strip()

        if not billno and not customer_name:
            messagebox.showwarning("Input Error", "Please enter either Bill Number or Customer Name")
            return

        try:
            found = False

            if billno:
                cursor.execute("SELECT CustomerName, PhoneNo, Address FROM Customers WHERE BillNumber = ?", (billno,))
                row = cursor.fetchone()
                if row:
                    name_var.set(row[0])
                    phone_var.set(row[1])
                    address_var.set(row[2])
                    found = True

            if not found and customer_name:
                cursor.execute("SELECT BillNumber, PhoneNo, Address FROM Customers WHERE LOWER(CustomerName) LIKE ?", (f"%{customer_name.lower()}%",))
                row = cursor.fetchone()
                if row:
                    billno_var.set(row[0])
                    phone_var.set(row[1])
                    address_var.set(row[2])
                    name_var.set(customer_name)
                    found = True

            if not found:
                messagebox.showerror("Not Found", "No customer found with the given details.")
                billno_var.set("")
                name_var.set("")
                phone_var.set("")
                address_var.set("")

        except Exception as e:
            messagebox.showerror("Database Error", f"Error fetching data: {e}")


    # --- Live Search Popup for Customer Name ---
    def show_customer_suggestions(event):
        typed = name_var.get().strip()
        if not typed:
            if hasattr(root, 'cust_popup'):
                root.cust_popup.destroy()
            return

        try:
            # Search customers starting with typed letters (case-insensitive)
            cursor.execute("SELECT CustomerName FROM Customers WHERE LOWER(CustomerName) LIKE ? ORDER BY CustomerName ASC", (typed.lower() + '%',))
            results = cursor.fetchall()
            names = [row[0] for row in results]

            if not names:
                if hasattr(root, 'cust_popup'):
                    root.cust_popup.destroy()
                return

            # Destroy old popup if exists
            if hasattr(root, 'cust_popup'):
                root.cust_popup.destroy()

            popup = tk.Toplevel(root)
            root.cust_popup = popup
            popup.wm_overrideredirect(True)  # No window decorations
            x = name_entry.winfo_rootx()
            y = name_entry.winfo_rooty() + name_entry.winfo_height()
            popup.geometry(f"+{x}+{y}")

            listbox = tk.Listbox(popup, font=("arial", 12), width=30, height=min(len(names), 10))
            listbox.pack()

            for name in names:
                listbox.insert(tk.END, name)

            def on_select(event):
                if listbox.curselection():
                    selected_name = listbox.get(listbox.curselection())
                    name_var.set(selected_name)

                    # Fetch details for selected customer name
                    try:
                        cursor.execute("SELECT BillNumber, PhoneNo, Address FROM Customers WHERE CustomerName = ?", (selected_name,))
                        row = cursor.fetchone()
                        if row:
                            billno_var.set(row[0])
                            phone_var.set(row[1])
                            address_var.set(row[2])
                        else:
                            billno_var.set("")
                            phone_var.set("")
                            address_var.set("")
                    except Exception as e:
                        messagebox.showerror("Database Error", f"Error fetching data: {e}")

                    popup.destroy()
                    name_entry.focus_set()

            listbox.bind("<<ListboxSelect>>", on_select)

            # Close popup when focus lost
            popup.bind("<FocusOut>", lambda e: popup.destroy())

        except Exception as e:
            if hasattr(root, 'cust_popup'):
                root.cust_popup.destroy()

    name_var = tk.StringVar()
    # Define name_entry BEFORE you bind events to it
    name_entry = Entry(cust_frame, font=("arial", 15), bd=3, relief=GROOVE, textvariable=name_var)
    name_entry.grid(row=0, column=1, padx=10, pady=5)
    
    # Then bind the event after name_entry is defined
    name_entry.bind("<KeyRelease>", show_customer_suggestions)


    # ---------- Search Button ----------
    tk.Button(cust_frame, text="Search", font=("Arial", 12, "bold"), bg="white", fg="#0f4d7d",
              command=fetch_customer_details).grid(row=0, column=6, padx=(5, 10), pady=5)

    # ---------- Cash Discount Section Frame (to keep items aligned together) ----------
    cash_disc_frame = tk.Frame(cust_frame, bg="#2c3e50")
    cash_disc_frame.grid(row=1, column=7, columnspan=3, sticky="w", padx=(10, 2), pady=5)
    
    # ---------- Cash Discount Label ----------
    tk.Label(cash_disc_frame, text="Cash Disc.:", bg="#2c3e50", fg="white",
             font=("Segoe UI", 13, "bold")).grid(row=0, column=0, sticky="e")
    
    # ---------- Cash Discount Entry ----------
    cash_disc_var = tk.StringVar()
    entry_disc = ttk.Entry(cash_disc_frame, textvariable=cash_disc_var, width=6,font=("Segoe UI",10, "bold"))
    entry_disc.grid(row=0, column=1, padx=(5, 2), ipady=3)
    
    # ---------- % Checkbox ----------
    is_percent = tk.BooleanVar(value=True)
    percent_cb = ttk.Checkbutton(cash_disc_frame, text="%", variable=is_percent)
    percent_cb.grid(row=0, column=2, padx=(2, 2))
    
    
    def apply_discount():
        disc_val = cash_disc_var.get().strip()
        percent_mode = is_percent.get()
    
        try:
            disc_val = float(disc_val)
        except ValueError:
            tk.messagebox.showerror("Invalid Discount", "Please enter a valid discount value.")
            return
    
        total_discount = 0.0
    
        for row in all_entry_rows:
            try:
                list_price = float(row[6].get().strip())
            except:
                list_price = 0
            try:
                net_qty = float(row[5].get().strip())
            except:
                net_qty = 0
    
            base_price = list_price * net_qty
    
            if percent_mode:
                discount_amt = base_price * (disc_val / 100)
            else:
                discount_amt = disc_val
    
            # Ensure flat discount doesn't exceed base price
            if not percent_mode and discount_amt > base_price:
                discount_amt = base_price
    
            new_gross_price = base_price - discount_amt
            new_unit_price = new_gross_price / net_qty if net_qty else 0
    
            row[8].delete(0, tk.END)
            row[8].insert(0, f"{new_unit_price:.2f}")
    
            row[9].delete(0, tk.END)
            row[9].insert(0, f"{new_gross_price:.2f}")
    
            total_discount += discount_amt
    
        # Update total LP value again
        update_lp_gross_and_total()
    
        # Update summary section discount
        net_discount_var.set(f"{total_discount:.2f}")
    
    apply_btn = ttk.Button(cash_disc_frame, text="✔", width=2, command=apply_discount)
    apply_btn.grid(row=0, column=3, padx=(5, 0))


    # ---------- Row 0 ----------
    tk.Label(cust_frame, text="Phone No.:", bg="#2c3e50", fg="white",
             font=("Segoe UI", 15, "bold")).grid(row=0, column=2, sticky="e", padx=(10, 2), pady=5)
    entry_phone = ttk.Entry(cust_frame, width=22,textvariable=phone_var,font=("Segoe UI", 10, "bold"))
    entry_phone.grid(row=0, column=3, padx=(2, 15), pady=5, ipady=4)
    
    tk.Label(cust_frame, text="Bill Number:", bg="#2c3e50", fg="white",
             font=("Segoe UI", 15, "bold")).grid(row=0, column=4, sticky="e", padx=(5, 2), pady=5)
    entry_bill = ttk.Entry(cust_frame, width=22, textvariable=billno_var,font=("Segoe UI", 10, "bold"))
    entry_bill.grid(row=0, column=5, padx=(2, 15), pady=5, ipady=4)

    # ---------- Cancel Reason Label + Combobox-as-Entry ----------
    tk.Label(cust_frame, text="Remarks:", bg="#2c3e50", fg="white",
             font=("Segoe UI", 15, "bold")).grid(row=0, column=7, sticky="e", padx=(0, 0), pady=5)
    
    cancel_reasons = [
        "Customer Cancelled",
        "Out of Stock",
        "Incorrect Order",
        "Delayed Delivery",
        "Other"
    ]
    
    cancel_reason_var = tk.StringVar()
    entry_cancel_reason = ttk.Combobox(cust_frame, textvariable=cancel_reason_var, values=cancel_reasons, width=20,font=("Segoe UI", 10, "bold"))
    entry_cancel_reason.grid(row=0, column=8, padx=(0, 0), pady=5, ipady=4)
    entry_cancel_reason.set("")  # Start blank
    entry_cancel_reason.configure(state="normal")  # Editable like Entry
    
 
    
    # ---------- Row 1 ----------
    tk.Label(cust_frame, text="Shipping Address:", bg="#2c3e50", fg="white",
             font=("Segoe UI", 15, "bold")).grid(row=1, column=0, sticky="e", padx=(5, 2), pady=5)
    entry_address = ttk.Entry(cust_frame, width=45, textvariable=address_var,font=("Segoe UI", 10, "bold"))
    entry_address.grid(row=1, column=1, columnspan=2, padx=(2, 15), pady=5, ipady=4)
    
    tk.Label(cust_frame, text="Bill Date:", bg="#2c3e50", fg="white",
             font=("Segoe UI", 15, "bold")).grid(row=1, column=3, sticky="e", padx=(5, 2), pady=5)
    entry_bill_date = DateEntry(cust_frame, width=20, background='darkblue',
                                foreground='white', borderwidth=2, date_pattern='dd/mm/yyyy',font=("Segoe UI", 10, "bold"))
    entry_bill_date.set_date(datetime.today())
    entry_bill_date.grid(row=1, column=4, padx=(2, 15), pady=5, ipady=4)
    
    tk.Label(cust_frame, text="Delivery Date:", bg="#2c3e50", fg="white",
             font=("Segoe UI", 15, "bold")).grid(row=1, column=5, sticky="e", padx=(5, 2), pady=5)
    entry_delivery = DateEntry(cust_frame, width=20, background='darkblue',
                               foreground='white', borderwidth=2, date_pattern='dd/mm/yyyy',font=("Segoe UI", 10, "bold"))
    entry_delivery.set_date(datetime.today())
    entry_delivery.grid(row=1, column=6, padx=(2, 10), pady=5, ipady=4)
    
    # ---------- Initialize Globals First (before any function uses them) ----------
    current_row = 1
    popup_window = None
    all_entry_rows = []
    lp_total_var = tk.StringVar(value="0.00")
    
    # ---------- Scrollable Canvas Setup ----------
    scroll_frame = tk.Frame(root)
    scroll_frame.pack(fill="both", expand=True, padx=10, pady=5)
    
    canvas = tk.Canvas(scroll_frame, bg="white", highlightthickness=0)
    canvas.pack(side="left", fill="both", expand=True)
    
    v_scroll = tk.Scrollbar(scroll_frame, orient="vertical", command=canvas.yview)
    v_scroll.pack(side="right", fill="y")
    
    h_scroll = tk.Scrollbar(root, orient="horizontal", command=canvas.xview)
    h_scroll.pack(fill="x")
    
    canvas.configure(yscrollcommand=v_scroll.set, xscrollcommand=h_scroll.set)
    
    # ---------- Combined Frame inside Canvas (with header + table) ----------
    container_frame = tk.Frame(canvas, bg="white")  # New master frame
    canvas.create_window((0, 0), window=container_frame, anchor="nw")
    
    summary_frame1 = tk.Frame(container_frame, bg="white")  # Header inside canvas
    summary_frame1.pack(fill="x")
    
    table_frame = tk.Frame(container_frame, bg="white")
    table_frame.pack(fill="both", expand=True)
    
    # ---------- Make canvas scroll properly ----------
    def _on_mousewheel(event):
        canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")
    
    def _on_shift_mousewheel(event):
        canvas.xview_scroll(int(-1 * (event.delta / 120)), "units")
    
    container_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
    
    canvas.bind("<Enter>", lambda e: canvas.bind_all("<MouseWheel>", _on_mousewheel))
    canvas.bind("<Leave>", lambda e: canvas.unbind_all("<MouseWheel>"))
    
    canvas.bind("<Enter>", lambda e: canvas.bind_all("<Shift-MouseWheel>", _on_shift_mousewheel))
    canvas.bind("<Leave>", lambda e: canvas.unbind_all("<Shift-MouseWheel>"))

    
    cols = [
        "DESCRIPTION", "RATINGS", "CAT NO.", "MAKE",
        "QUANTITY", "NET QTY", "LIST PRICE", "DISCOUNT",
        "DISC UNIT PRICE", "DISC GROSS PRICE", "LP GRASS.PRICE"
    ]
    
    entry_font = ("Helvetica", 10)
    header_font = ("Helvetica", 10, "bold")
    header_bg = "#2e3f50"
    header_fg = "white"
    
    for j in range(len(cols)):
        summary_frame1.grid_columnconfigure(j, weight=1, uniform="col")
        table_frame.grid_columnconfigure(j, weight=1, uniform="col")
    
    for j, col in enumerate(cols):
        header = tk.Label(summary_frame1, text=col, font=header_font,
                          bg=header_bg, fg=header_fg, relief="ridge",
                          padx=2, pady=4)
        header.grid(row=0, column=j, sticky="nsew")
    
    # ---------- Utility Functions ----------
    def update_lp_gross_and_total():
        total = 0.0
        for row in all_entry_rows:
            try: qty = float(row[4].get().strip())
            except: qty = 0
            try: price = float(row[6].get().strip())
            except: price = 0
    
            lp_value = qty * price
            row[10].delete(0, tk.END)
            row[10].insert(0, f"{lp_value:.2f}")
            total += lp_value
    
        lp_total_var.set(f"{total:.2f}")
    
    def get_mode_quantity(description):
        quantities = []
        for row in all_entry_rows:
            desc = row[0].get().strip()
            qty = row[4].get().strip()
            if desc == description and qty.isdigit():
                quantities.append(qty)
        if quantities:
            count = Counter(quantities)
            mode = count.most_common(1)[0][0]
            return mode
        return ""
    
    def update_net_qty_on_change(row_index):
        row_widgets = all_entry_rows[row_index]
        description = row_widgets[0].get().strip()
        mode_qty = get_mode_quantity(description)
        if mode_qty:
            row_widgets[5].delete(0, tk.END)
            row_widgets[5].insert(0, mode_qty)

        # ---------- Sample Inventory ----------
    inventory_data = [
        {"DESCRIPTION": "MCB 10A", "RATINGS": "10A", "CAT NO.": "M10", "MAKE": "Havells",   "QUANTITY": "1", "LIST PRICE": "120.00"},
        {"DESCRIPTION": "MCB 16A", "RATINGS": "16A", "CAT NO.": "M16", "MAKE": "Siemens",   "QUANTITY": "1", "LIST PRICE": "130.00"},
        {"DESCRIPTION": "RCCB 40A", "RATINGS": "40A", "CAT NO.": "R40", "MAKE": "Anchor",    "QUANTITY": "1", "LIST PRICE": "450.00"},
        {"DESCRIPTION": "MCB 6A", "RATINGS": "6A", "CAT NO.": "M6", "MAKE": "Schneider", "QUANTITY": "1", "LIST PRICE": "110.00"},
        {"DESCRIPTION": "ELCB 63A", "RATINGS": "63A", "CAT NO.": "E63", "MAKE": "Havells",   "QUANTITY": "1", "LIST PRICE": "650.00"},
    ]

    
    def show_inventory_popup(entry_widget, row_index):
        global popup_window
    
        if popup_window:
            popup_window.destroy()
    
        typed = entry_widget.get().strip().lower()
        matches = [item for item in inventory_data if typed in item["DESCRIPTION"].lower()]
        if not matches:
            return
    
        x = entry_widget.winfo_rootx()
        y = entry_widget.winfo_rooty() + entry_widget.winfo_height()
    
        popup_window = tk.Toplevel()
        popup_window.geometry(f"700x200+{x}+{y}")
        popup_window.wm_overrideredirect(True)
    
        frame = tk.Frame(popup_window)
        frame.pack(fill="both", expand=True)
    
        tree_scroll = tk.Scrollbar(frame)
        tree_scroll.pack(side="right", fill="y")
    
        tree = ttk.Treeview(frame, columns=("DESCRIPTION", "RATINGS", "CAT NO.", "MAKE", "QUANTITY", "LIST PRICE"),
                            show="headings", height=6, yscrollcommand=tree_scroll.set)
        tree.pack(fill="both", expand=True)
        tree_scroll.config(command=tree.yview)
    
        for col in ("DESCRIPTION", "RATINGS", "CAT NO.", "MAKE", "QUANTITY", "LIST PRICE"):
            tree.heading(col, text=col)
            tree.column(col, width=100, anchor="center")
    
        for item in matches:
            values = (item["DESCRIPTION"], item["RATINGS"], item["CAT NO."], item["MAKE"], item["QUANTITY"], item["LIST PRICE"])
            tree.insert("", tk.END, values=values)
    
        def on_row_double_click(event):
            selected = tree.focus()
            if selected:
                values = tree.item(selected, "values")
                row_widgets = all_entry_rows[row_index]
                for i in range(5):
                    row_widgets[i].delete(0, tk.END)
                    row_widgets[i].insert(0, values[i])
                row_widgets[6].delete(0, tk.END)
                row_widgets[6].insert(0, values[5])
                update_lp_gross_and_total()
                update_net_qty_on_change(row_index)
            popup_window.destroy()
    
        tree.bind("<Double-1>", on_row_double_click)

    def delete_row(row_index):
        if row_index >= len(all_entry_rows):
            print(f"Cannot delete row at index {row_index}: out of range.")
            return
    
        for widget in all_entry_rows[row_index]:
            widget.destroy()
    
        del all_entry_rows[row_index]
        
        update_lp_gross_and_total()
    
    def on_discount_backspace(event, idx):
        if 0 <= idx < len(all_entry_rows):
            delete_row(idx)
        else:
            print(f"Invalid index {idx} for deletion.")

        
    # ---------- Entry Row ----------
    def insert_entry_row(data=[]):
        global current_row, all_entry_rows
        row_widgets = []
        row_index = len(all_entry_rows)
    
        for j in range(len(cols)):
            value = data[j] if j < len(data) else ""
            e = tk.Entry(table_frame, font=entry_font, justify="center", bg="white", relief="solid", borderwidth=1)
            e.grid(row=current_row, column=j, sticky="nsew", padx=1, pady=2, ipady=3)

            e.insert(0, value)
            row_widgets.append(e)
    
            # NOTE: This is now handled globally at the start
            # table_frame.grid_columnconfigure(j, weight=1)
    
            if cols[j] == "DESCRIPTION":
                e.bind("<KeyRelease>", lambda event, ent=e, idx=row_index: show_inventory_popup(ent, idx))
    
            if cols[j] == "QUANTITY":
                def on_qty_change(event, idx=row_index):
                    update_net_qty_on_change(idx)
                    update_lp_gross_and_total()
                e.bind("<KeyRelease>", on_qty_change)
    
            if cols[j] == "LIST PRICE":
                e.bind("<KeyRelease>", lambda event: update_lp_gross_and_total())
    
            if cols[j] == "DISCOUNT":
                def on_discount_tab(event, row_idx=row_index):
                    entry = event.widget
                    discount_str = entry.get().strip().replace('%', '')
                    try:
                        discount = float(discount_str)
                    except:
                        discount = 0
                    entry.delete(0, tk.END)
                    entry.insert(0, f"{discount}%")
    
                    row_widgets = all_entry_rows[row_idx]
                    try:
                        list_price = float(row_widgets[6].get().strip())
                    except:
                        list_price = 0
                    try:
                        net_qty = float(row_widgets[5].get().strip())
                    except:
                        net_qty = 0
    
                    disc_unit_price = list_price - (list_price * discount / 100)
                    disc_gross_price = disc_unit_price * net_qty
    
                    row_widgets[8].delete(0, tk.END)
                    row_widgets[8].insert(0, f"{disc_unit_price:.2f}")
                    row_widgets[9].delete(0, tk.END)
                    row_widgets[9].insert(0, f"{disc_gross_price:.2f}")
    
                    update_lp_gross_and_total()
                    insert_entry_row()
                    return "break"
    
                def on_discount_return(event):
                    event.widget.tk_focusNext().focus()
                    insert_section_heading_input()
                    return "break"
    
                e.bind("<Tab>", on_discount_tab)
                e.bind("<Return>", on_discount_return)
                e.bind("<KeyRelease-BackSpace>", lambda event, i=row_index: on_discount_backspace(event, i))
    
        all_entry_rows.append(row_widgets)
        current_row += 1

   
    # ---------- Section Heading ----------
    def insert_section_heading_input():
        global current_row
        entry = tk.Entry(table_frame, font=("Helvetica", 11, "bold"), bg="#d0e0f0", justify="left")
        entry.grid(row=current_row, column=0, columnspan=len(cols), sticky="nsew", pady=(8, 4), ipady=3)
        entry.focus()
        entry.bind("<Return>", process_section_heading)
    
    def process_section_heading(event):
        global current_row
        heading_text = event.widget.get().strip()
        if heading_text:
            event.widget.grid_forget()
            label = tk.Label(table_frame, text=heading_text, font=("Helvetica", 11, "bold"),
                             bg="#d0e0f0", anchor="w", padx=10, pady=6, relief="ridge")
            label.grid(row=current_row, column=0, columnspan=len(cols), sticky="nsew", pady=(8, 4))
            current_row += 1
            insert_entry_row()
    
    # ---------- Start ----------
    insert_section_heading_input()
    # ----------------- Bill Summary Frame -----------------
    summary_frame = tk.LabelFrame(root, text="Bill Menu", font=("Segoe UI", 11, "bold"),
                                  bg="#2c3e50", fg="white", padx=10, pady=10)
    summary_frame.pack(fill="x", padx=10, pady=5)


    enclosure_total_var = tk.StringVar(value="₹ 0.00")
    
    def open_enclosure(total_var=None):
        popup = tk.Toplevel()
        popup.title("Enclosure Fabrication")
        popup.geometry("1100x700")
        popup.configure(bg="white")
    
        headers = ["✔", " ", "MM", "Length", "Width", "", "Kg", "", "Kg per Sqmm"]
    
        enclosure_data = [
            {" ": "18 SWG", "MM": 1.20, "Length": 2500, "Width": 1250, "Kg": 30, "": 29.4375, "Kg per Sqmm": 0.0000096},
            {" ": "16 SWG", "MM": 1.50, "Length": 2500, "Width": 1250, "Kg": 40, "": 36.796875, "Kg per Sqmm": 0.0000128},
            {" ": "14 SWG", "MM": 2.00, "Length": 2500, "Width": 1250, "Kg": 50, "": 49.0625, "Kg per Sqmm": 0.000016},
            {" ": "13 SWG", "MM": 2.30, "Length": 2500, "Width": 1250, "Kg": 60, "": 56.421875, "Kg per Sqmm": 0.0000192},
            {" ": "12 SWG", "MM": 2.64, "Length": 2500, "Width": 1250, "Kg": 70, "": 64.7625, "Kg per Sqmm": 0.0000224},
            {" ": "11 SWG", "MM": 3.00, "Length": 2500, "Width": 1250, "Kg": 80, "": 73.59375, "Kg per Sqmm": 0.0000256},
            {" ": "10 SWG", "MM": 3.25, "Length": 2500, "Width": 1250, "Kg": 90, "": 79.7265625, "Kg per Sqmm": 0.0000288},
        ]
    
        entry_matrix1 = []
        checkbox_vars = []
    
        def update_total(row_idx):
            try:
                mm = float(entry_matrix1[row_idx][1].get())
                length = float(entry_matrix1[row_idx][2].get())
                width = float(entry_matrix1[row_idx][3].get())
                total = round(mm * length * width * 0.00000785, 2)
                entry_matrix1[row_idx][4].config(state='normal')
                entry_matrix1[row_idx][4].delete(0, tk.END)
                entry_matrix1[row_idx][4].insert(0, total)
                entry_matrix1[row_idx][4].config(state='readonly', disabledforeground="black")
            except ValueError:
                pass
    
        def format_decimal(value):
            return "{:.10f}".format(value).rstrip("0").rstrip(".") if "." in "{:.10f}".format(value) else "{:.10f}".format(value)
        
        def update_kg_per_sqmm(row_idx):
            try:
                kg = float(entry_matrix1[row_idx][5].get() or 0)
                length = float(entry_matrix1[row_idx][2].get() or 0)
                width = float(entry_matrix1[row_idx][3].get() or 0)
        
                if length > 0 and width > 0:
                    result = kg / (length * width)
                    formatted_result = format_decimal(result)
                else:
                    formatted_result = "0.0000000000"
        
                entry_matrix1[row_idx][8].config(state='normal')
                entry_matrix1[row_idx][8].delete(0, tk.END)
                entry_matrix1[row_idx][8].insert(0, formatted_result)
                entry_matrix1[row_idx][8].config(state='readonly', disabledforeground="black")
        
            except Exception:
                entry_matrix1[row_idx][8].config(state='normal')
                entry_matrix1[row_idx][7].delete(0, tk.END)
                entry_matrix1[row_idx][7].insert(0, "Error")
                entry_matrix1[row_idx][7].config(state='readonly', disabledforeground="red")
        
        popup.grid_columnconfigure(0, minsize=30)
        
        # Header labels for first table
        for col, text in enumerate(headers):
            lbl = tk.Label(popup, text=text if text else "Calculated",
                           font=('times new roman', 12, "bold"),
                           bg="#A1AEB1", fg="black", width=14, borderwidth=1, relief="solid")
            lbl.grid(row=0, column=col, padx=1, pady=8, sticky="nsew")
        
        # Populate first table rows
        for row_index, data in enumerate(enclosure_data):
            row_entries = []
            var = tk.IntVar()
            checkbox_vars.append(var)
        
            # Checkbox
            chk = tk.Checkbutton(popup, variable=var, bg="white", padx=0, pady=0, takefocus=0)
            chk.grid(row=row_index + 1, column=0, padx=1, pady=3, sticky="w")
            row_entries.append(None)
        
            for col_index, header in enumerate(headers[1:], start=1):
                ent = tk.Entry(popup, font=('times new roman', 12), width=14,
                               borderwidth=1, relief="solid", justify='center')
                ent.grid(row=row_index + 1, column=col_index, padx=1, pady=3, sticky="nsew")
        
                # Fill initial values or calculate
                if header in data:
                    ent.insert(0, data[header])
                elif col_index == 4:  # MM*L*W*0.00000785
                    mm = float(data.get("MM", 0))
                    length = float(data.get("Length", 0))
                    width = float(data.get("Width", 0))
                    total = round(mm * length * width * 0.00000785, 2)
                    ent.insert(0, total)
                    ent.config(state='readonly', disabledforeground="black")
                elif col_index == 7:  # Kg / (L * W)
                    kg = float(data.get("Kg", 0))
                    length = float(data.get("Length", 0))
                    width = float(data.get("Width", 0))
                    if length > 0 and width > 0:
                        result = kg / (length * width)
                        formatted_result = format_decimal(result)
                    else:
                        formatted_result = "0.0000000000"
                    ent.insert(0, formatted_result)
                    ent.config(state='readonly', disabledforeground="black")
        
                # Bind to recalculate when Kg, Length, or Width changes
                if header in ["Length", "Width", "Kg"]:
                    ent.bind("<FocusOut>", lambda e, r=row_index: update_kg_per_sqmm(r))
                    ent.bind("<Return>", lambda e, r=row_index: update_kg_per_sqmm(r))
                    ent.bind("<Tab>", lambda e, r=row_index: update_kg_per_sqmm(r))
        
                # Optional: Highlight certain rows
                if row_index in [0, 2, 4, 6]:
                    ent.config(bg="#FF7F7F")
        
                row_entries.append(ent)
            entry_matrix1.append(row_entries)



    
        # Second table headers and initial data
        headers1 = ["", "L", "W", "H", "Volume"]
        second_table_row_start = len(enclosure_data) + 3
        col_width = 14
    
        header_bg = "#FFD700"
        header_fg = "#4B4B4B"
    
        Enclosure_data1 = [
            {"": "28+1+1", "L": 2400, "W": 300, "H": 1600, "Volume": ""},
        ]
    
        autofill_map = {
            1: [1, 3],  # L column autofill rows
            2: [3],     # W column autofill rows
        }
    
        fallback_values = {
            (2, 1): 300,
            (1, 2): 1600,
            (2, 2): 1600,
        }
    
        entry_matrix2 = []
    
        def smart_autofill(col_index, value):
            if col_index in [3, 4]:  # Skip autofill for H and Volume
                return
    
            filled_rows = set()
            for row in autofill_map.get(col_index, []):
                entry_matrix2[row][col_index].delete(0, tk.END)
                entry_matrix2[row][col_index].insert(0, value)
                filled_rows.add(row)
    
            for row in range(1, 5):
                if (row, col_index) not in filled_rows:
                    cell = entry_matrix2[row][col_index]
                    if not cell.get().strip():
                        fallback = fallback_values.get((row, col_index))
                        if fallback is not None:
                            cell.insert(0, fallback)
    
        # Second table header row
        for col, text in enumerate(headers1):
            lbl = tk.Label(popup, text=text, font=('times new roman', 12, "bold"),
                           bg=header_bg, fg=header_fg, width=col_width, borderwidth=1, relief="solid")
            lbl.grid(row=second_table_row_start, column=col, padx=1, pady=(15, 0), sticky="nsew")
    
        # Second table entry grid
        for row in range(6):
            row_entries = []
            for col, header in enumerate(headers1):
                bg_color = header_bg if row == 0 else "white"
                ent = tk.Entry(popup, font=('times new roman', 12), bg=bg_color, width=col_width,
                               borderwidth=1, relief="solid", justify='center')
                ent.grid(row=second_table_row_start + row + 1, column=col, padx=1, pady=3, sticky="nsew")
    
                if row == 0:
                    ent.insert(0, Enclosure_data1[0].get(header, ""))
                    def make_callback(c=col):
                        def on_focus_out(event):
                            smart_autofill(c, event.widget.get())
                        return on_focus_out
                    ent.bind("<FocusOut>", make_callback())
                elif col == 0 and 1 <= row <= 5:
                    ent.insert(0, str(row))  # serial numbers 1-5
    
                row_entries.append(ent)
            entry_matrix2.append(row_entries)
    
        # Total calculation function
        def enclosure_calculate_total():
            total_volume = 0
            for row in entry_matrix2[1:7]:
                try:
                    l = float(row[1].get())
                    w = float(row[2].get())
                    h = float(row[3].get())
                    volume = l * w * h
                    total_volume += volume
                    row[4].config(state='normal')
                    row[4].delete(0, tk.END)
                    row[4].insert(0, f"{int(volume)}")
                    row[4].config(state='readonly', disabledforeground="black")
                except ValueError:
                    row[4].config(state='normal')
                    row[4].delete(0, tk.END)
                    row[4].config(state='readonly')
            enclosure_total_var.set(str(int(total_volume)))
    
        btn_row = second_table_row_start + 7
    
        # Percent section (left aligned)
        percent_label = tk.Label(popup, text="Percent:", font=('times new roman', 12, "bold"), bg="white")
        percent_label.grid(row=btn_row - 1, column=0, pady=8, padx=(15,5), sticky="w")
    
        percent_entry = tk.Entry(popup, font=('times new roman', 12), width=18)
        percent_entry.grid(row=btn_row - 1, column=1, pady=8, padx=5, sticky="w")
    
        percent_result_entry = tk.Entry(popup, font=('times new roman', 12), width=18, state='readonly',
                                       disabledforeground="black")
        percent_result_entry.grid(row=btn_row - 1, column=2, pady=8, padx=5, sticky="w")
    
        percent_final_entry = tk.Entry(popup, font=('times new roman', 12), width=18, state='readonly',
                                      disabledforeground="black")
        percent_final_entry.grid(row=btn_row - 1, column=3, pady=8, padx=5, sticky="w")
    
        # Buttons aligned nicely below
        tk.Button(popup, text="Total", command=enclosure_calculate_total,
                  font=('times new roman', 12, "bold"), bg="#0f4d7d", fg="white")\
            .grid(row=btn_row, column=1, pady=12, padx=15, sticky="w")
    
        tk.Button(popup, text="Close", command=popup.destroy,
                  font=('times new roman', 12, "bold"), bg="#0f4d7d", fg="white")\
            .grid(row=btn_row, column=0, pady=12, padx=15, sticky="e")
    
        def apply_percent_to_total(event=None):
            try:
                enclosure_calculate_total()
                percent_text = percent_entry.get().strip()
                if not percent_text:
                    percent_result_entry.config(state='normal')
                    percent_result_entry.delete(0, tk.END)
                    percent_result_entry.config(state='readonly')
                    percent_final_entry.config(state='normal')
                    percent_final_entry.delete(0, tk.END)
                    percent_final_entry.config(state='readonly')
                    return
    
                if not percent_text.endswith("%"):
                    percent_text += "%"
    
                percent_entry.delete(0, tk.END)
                percent_entry.insert(0, percent_text)
    
                percent = float(percent_text.rstrip("%"))
                total_volume = float(enclosure_total_var.get())
    
                result = (percent / 100) * total_volume
                final = total_volume + result
    
                percent_result_entry.config(state='normal')
                percent_result_entry.delete(0, tk.END)
                percent_result_entry.insert(0, f"{round(result, 2)}")
                percent_result_entry.config(state='readonly')
    
                percent_final_entry.config(state='normal')
                percent_final_entry.delete(0, tk.END)
                percent_final_entry.insert(0, f"{round(final, 2)}")
                percent_final_entry.config(state='readonly')
    
            except Exception:
                percent_result_entry.config(state='normal')
                percent_result_entry.delete(0, tk.END)
                percent_result_entry.config(state='readonly')
    
                percent_final_entry.config(state='normal')
                percent_final_entry.delete(0, tk.END)
                percent_final_entry.config(state='readonly')
    
        percent_entry.bind("<FocusOut>", apply_percent_to_total)
        percent_entry.bind("<Return>", apply_percent_to_total)
    
        # Initial total calculation
        enclosure_calculate_total()

    

        



    wiring_total_var = tk.StringVar(value="₹ 0.00")
    
    def open_wiring(total_var=None):
        popup = tk.Toplevel()
        popup.title("Wiring + Accessories")
        popup.geometry("850x460")  # Fits everything without scroll
        popup.configure(bg="#eef2f3")
        popup.resizable(False, False)
    
        headers = ["SIZE", "RATE", "QTY", "TOTAL"]
    
        predefined_items = [
            (35, 350),
            (25, 250),
            (16, 170),
            (10, 110),
            (6, 60),
            (4, 40),
            (2.5, 30),
            (1, 15)
        ]
    
        # Main frame without scrollbars
        frame = tk.Frame(popup, bg="#eef2f3")
        frame.pack(padx=15, pady=15)
    
        # Table Header
        for col, text in enumerate(headers):
            lbl = tk.Label(
                frame,
                text=text,
                font=('Calibri', 12, "bold"),
                bg="#0f4d7d",
                fg="white",
                width=20,
                pady=8,
                bd=1,
                relief="solid"
            )
            lbl.grid(row=0, column=col, padx=1, pady=1)
    
        entry_matrix = []
    
        def wiring_calculate_total():
            total = 0
            for row in entry_matrix:
                try:
                    qty = float(row[2].get())
                    rate = float(row[1].get())
                    row_total = qty * rate
                    row[3].config(state="normal")
                    row[3].delete(0, tk.END)
                    row[3].insert(0, f"{row_total:.2f}")
                    row[3].config(state="readonly")
                    total += row_total
                except:
                    row[3].config(state="normal")
                    row[3].delete(0, tk.END)
                    row[3].insert(0, "")
                    row[3].config(state="readonly")
                    continue
            wiring_total_var.set(f"₹ {total:.2f}")
    
        def wiring_add_row(size="", rate=""):
            row_idx = len(entry_matrix) + 1
            entries = []
            bg_color = "#ffffff" if row_idx % 2 == 0 else "#f1f9ff"
    
            e1 = tk.Entry(frame, width=20, bg=bg_color, font=('Calibri', 11), relief="solid", bd=1)
            e1.insert(0, size)
            e1.grid(row=row_idx, column=0, padx=1, pady=2)
            entries.append(e1)
    
            e2 = tk.Entry(frame, width=20, bg=bg_color, font=('Calibri', 11), relief="solid", bd=1)
            e2.insert(0, rate)
            e2.grid(row=row_idx, column=1, padx=1, pady=2)
            entries.append(e2)
    
            e3 = tk.Entry(frame, width=20, bg=bg_color, font=('Calibri', 11), relief="solid", bd=1)
            e3.grid(row=row_idx, column=2, padx=1, pady=2)
            entries.append(e3)
    
            e4 = tk.Entry(frame, width=20, state="readonly", readonlybackground="#e0f0ff",
                          font=('Calibri', 11), relief="solid", bd=1)
            e4.grid(row=row_idx, column=3, padx=1, pady=2)
            entries.append(e4)
    
            def on_enter(event):
                wiring_calculate_total()
                if entries == entry_matrix[-1]:
                    wiring_add_row()
    
            e3.bind("<Return>", on_enter)
            entry_matrix.append(entries)
    
        for size, rate in predefined_items:
            wiring_add_row(size, rate)
    
        # Bottom Buttons
        btn_frame = tk.Frame(popup, bg="#eef2f3")
        btn_frame.pack(pady=15)
    
        button_style = {
            "font": ('Calibri', 12, "bold"),
            "bg": "#0f4d7d",
            "fg": "white",
            "activebackground": "#155a8a",
            "activeforeground": "white",
            "bd": 0,
            "width": 14,
            "cursor": "hand2"
        }
    
        tk.Button(btn_frame, text="Calculate Total", command=wiring_calculate_total, **button_style)\
            .grid(row=0, column=0, padx=10)
    
        tk.Button(btn_frame, text="Close", command=popup.destroy, **button_style)\
            .grid(row=0, column=1, padx=10)
        wiring_total_var = tk.StringVar(value="₹ 0.00")

        entry_wiring = ttk.Entry(summary_frame, textvariable=wiring_total_var, width=15)
        entry_wiring.grid(row=0, column=3, padx=5, pady=5)


    section_start_index = {}
    busbar_sections = {}
    def open_busbar():
        global table_frame, all_entries, busbar_total_label
    
        popup = tk.Toplevel()
        popup.title("BUS BAR")
        popup.geometry("1400x750")
        popup.configure(bg="white")
    
        canvas_frame = tk.Frame(popup, bg="white")
        canvas_frame.pack(fill=tk.BOTH, expand=True)
    
        canvas = tk.Canvas(canvas_frame, bg="white", highlightthickness=0)
        canvas.grid(row=0, column=0, sticky="nsew")
    
        scrollbar_x = ttk.Scrollbar(canvas_frame, orient="horizontal", command=canvas.xview)
        scrollbar_x.grid(row=1, column=0, sticky="ew")
        canvas.configure(xscrollcommand=scrollbar_x.set)
    
        canvas_frame.grid_rowconfigure(0, weight=1)
        canvas_frame.grid_columnconfigure(0, weight=1)
    
        scrollable_frame = tk.Frame(canvas, bg="white")
        window_id = canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    
        def on_frame_configure(event):
            canvas.configure(scrollregion=canvas.bbox("all"))
            canvas.itemconfig(window_id, width=max(scrollable_frame.winfo_reqwidth(), canvas.winfo_width()))
    
        scrollable_frame.bind("<Configure>", on_frame_configure)
    
        def _on_shiftmousewheel(event):
            canvas.xview_scroll(int(-1 * (event.delta / 120)), "units")
    
        canvas.bind_all("<Shift-MouseWheel>", _on_shiftmousewheel)
    
        headers = [
            "L", "W", "RUN", "WEIGHT (KG/M)", "PHASE", "NEUTRAL",
            "PANEL LENGTH", "NO. OF HORI", "TOTAL LENGTH (Mts)",
            "WEIGHT OF THE BUS BAR", "BUS BAR PRICE", "GRAND PRICE"
        ]
        num_cols = len(headers)
        current_row = 0
        all_entries = []
        material_density = tk.DoubleVar(value=0.0)
    
        style = ttk.Style()
        style.theme_use('clam')
        style.configure("CustomCombobox.TCombobox",
                        fieldbackground="white",
                        background="#0f4d7d",
                        foreground="black",
                        font=("Arial", 10),
                        padding=5)
    
        top_frame = tk.Frame(scrollable_frame, bg="#e6f2ff")
        top_frame.grid(row=current_row, column=0, sticky="ew", padx=10, pady=(5, 2))
        tk.Label(top_frame, text="BUS BAR", font=("Arial", 10, "bold"),
                 bg="#e6f2ff", fg="#003366").grid(row=0, column=1)
    
        selected_busbar = tk.StringVar()
        busbar_options = [
            "VERTICAL-INCOMER FOR B/C", "VERTICAL-O/G FOR Ph1", "VERTICAL-O/G FOR Ph2",
            "VERTICAL-APFC O/G", "VERTICAL-INCOMER", "VERTICAL-OUTGOING",
            "CONTROL BUS", "EARTH"
        ]
        
        combobox = ttk.Combobox(top_frame, textvariable=selected_busbar,
                                values=busbar_options,
                                state="readonly", width=30, style="CustomCombobox.TCombobox")
        combobox.set("Select Busbar Type")
        combobox.grid(row=0, column=1, sticky="w", padx=(20, 5))
        
        # New Entry + Add Button for user-defined busbars
        new_busbar_name = tk.StringVar()
        
        new_entry = tk.Entry(top_frame, textvariable=new_busbar_name, font=("Arial", 10), width=30)
        new_entry.grid(row=0, column=2, padx=5)
        
        def add_custom_busbar():
            name = new_busbar_name.get().strip()
            if not name:
                return
            if name in busbar_options:
                return  # Already exists
        
            busbar_options.append(name)
            combobox.config(values=busbar_options)
            create_section(name, [["" for _ in range(num_cols)]])
            new_busbar_name.set("")
        
        add_button = tk.Button(top_frame, text="Add", command=add_custom_busbar,
                               bg="#17a2b8", fg="white", font=("Arial", 9, "bold"))
        add_button.grid(row=0, column=3, padx=(5, 10))
        current_row += 1
    
        sub_frame = tk.Frame(scrollable_frame, bg="#d9fdd3")
        sub_frame.grid(row=current_row, column=0, sticky="ew", padx=10)
    
        def on_material_select(event):
            selection = sub_combobox.get()
            if "Aluminum" in selection:
                material_density.set(0.0027)
            elif "Copper" in selection:
                material_density.set(0.0089)
            else:
                material_density.set(0.0)
    
        sub_combobox = ttk.Combobox(sub_frame, state="readonly",
                                     values=["Aluminum = 0.0027", "Copper = 0.0089"],
                                     width=20, font=("Arial", 10),
                                     style="CustomCombobox.TCombobox")
        sub_combobox.set(" ")
        sub_combobox.bind("<<ComboboxSelected>>", on_material_select)
        sub_combobox.grid(row=0, column=0, sticky="w", padx=10)
    
        tk.Label(sub_frame, text="HORIZONTAL - MAIN TRFO & DG1",
                 font=("Arial", 10, "bold"), bg="#d9fdd3", fg="#004d00") \
            .grid(row=0, column=1, sticky="w", padx=10)
        current_row += 1
    
        busbar_data = {
            "VERTICAL-INCOMER FOR B/C": [["25", "10", "1", "", "3", "", "1600", "1", "", "", "400", ""]],
            "VERTICAL-O/G FOR Ph1": [["30", "5", "2", "", "1", "1", "1800", "2", "", "", "500", ""]],
            "VERTICAL-O/G FOR Ph2": [["40", "5", "2", "", "2", "1", "1500", "1", "", "", "520", ""]],
            "VERTICAL-APFC O/G": [["50", "10", "2", "", "1", "1", "1800", "1", "", "", "600", ""]],
            "VERTICAL-INCOMER": [["20", "10", "1", "", "1", "0", "1600", "1", "", "", "450", ""]],
            "VERTICAL-OUTGOING": [["20", "10", "1", "", "0", "1", "1600", "1", "", "", "450", ""]],
            "CONTROL BUS": [["10", "5", "1", "", "1", "1", "1600", "2", "", "", "300", ""]],
            "EARTH": [["16", "5", "1", "", "0", "1", "1600", "1", "", "", "200", ""]]
        }
    
        def on_last_cell_tab(event):
            widget = event.widget
            if all_entries and widget == all_entries[-1][-1]:
                add_new_row()
                return "break"
    
        def add_new_row():
            row_index = len(all_entries) + 1
            row_entries = []
            for c in range(num_cols):
                e = tk.Entry(table_frame, font=("Arial", 10), relief="solid", justify="center")
                e.grid(row=row_index, column=c, padx=1, pady=1, sticky="nsew")
                if c == num_cols - 1:
                    e.bind("<Tab>", on_last_cell_tab)
                row_entries.append(e)
            all_entries.append(row_entries)
    
        def create_section(title, data, deletable=False):
            nonlocal current_row
            global table_frame

            section_row = current_row

            section_title_frame = tk.Frame(scrollable_frame, bg="#728FCE")
            section_title_frame.grid(row=current_row, column=0, sticky="ew", padx=10, pady=(10, 2))

            section_label = tk.Label(section_title_frame, text=title, font=("Arial", 10, "bold"),
                                     bg="#728FCE", anchor="w")
            section_label.pack(side=tk.LEFT, fill=tk.X, expand=True)

            if deletable:
                def delete_section():
                    # Destroy the widgets
                    section_title_frame.destroy()
                    table_frame.destroy()

                    # Remove entries from all_entries
                    start_idx = section_start_index.get(title, 0)
                    num_rows = len(data)
                    del all_entries[start_idx:start_idx + num_rows]

                    # Remove from tracking dicts
                    if title in busbar_sections:
                        del busbar_sections[title]
                    if title in section_start_index:
                        del section_start_index[title]

                delete_button = tk.Button(F3, text="Delete Table", command=delete_table,
                          font=("times new roman", 12, "bold"),
                          bg="red", fg="white", padx=10, pady=5)
                delete_button.grid(row=0, column=2, padx=10)

            current_row += 1

            table_frame = tk.Frame(scrollable_frame, bg="white")
            table_frame.grid(row=current_row, column=0, sticky="ew", padx=10)

            for i in range(num_cols):
                table_frame.grid_columnconfigure(i, weight=1)

            for col, header in enumerate(headers):
                lbl = tk.Label(table_frame, text=header, font=("Arial", 9, "bold"),
                               bg="#ffe0b3", relief="solid", bd=1, width=14, anchor="center")
                lbl.grid(row=0, column=col, sticky="nsew", padx=1, pady=1)

            section_start_index[title] = len(all_entries)

            for r_idx, row_data in enumerate(data, start=1):
                row_entries = []
                for c in range(num_cols):
                    e = tk.Entry(table_frame, font=("Arial", 10), relief="solid", justify="center")
                    e.grid(row=r_idx, column=c, padx=1, pady=1, sticky="nsew")
                    if c < len(row_data):
                        e.insert(0, row_data[c])
                    if c == num_cols - 1:
                        e.bind("<Tab>", on_last_cell_tab)
                    row_entries.append(e)
                all_entries.append(row_entries)

            busbar_sections[title] = (section_title_frame, table_frame)
            current_row += 1
    
        def calc_all():
            density = material_density.get()
            for row in all_entries:
                try:
                    l = float(row[0].get() or 0)
                    w = float(row[1].get() or 0)
                    run = float(row[2].get() or 0)
                    weight_kgm = l * w * run * density
                    row[3].delete(0, tk.END)
                    row[3].insert(0, f"{weight_kgm:.4f}")
    
                    phase = int(row[4].get() or 0)
                    neutral = int(row[5].get() or 0)
                    panel_len = float(row[6].get() or 0)
                    no_of_hori = float(row[7].get() or 0)
                    total_len = (panel_len / 1000) * no_of_hori * (phase + neutral)
                    row[8].delete(0, tk.END)
                    row[8].insert(0, f"{total_len:.2f}")
    
                    weight_busbar = total_len * weight_kgm
                    row[9].delete(0, tk.END)
                    row[9].insert(0, f"{weight_busbar:.2f}")
    
                    price = float(row[10].get() or 0)
                    grand = weight_busbar * price
                    row[11].delete(0, tk.END)
                    row[11].insert(0, f"{grand:,.2f}")
                except Exception as e:
                    print("Row Error:", e)
    
        def show_grand_total():
            total = 0.0
            for row in all_entries:
                try:
                    val = float(row[11].get().replace(",", "") or 0)
                    total += val
                except:
                    pass
            busbar_total_var.set(f"{total:,.2f}")

    
        def on_combobox_select(event):
            selection = selected_busbar.get()
            if selection in busbar_data:
                create_section(selection, busbar_data[selection])


    
        combobox.bind("<<ComboboxSelected>>", on_combobox_select)
        busbar_total_var = tk.StringVar(value="0.00")  # Create a StringVar
    
        bottom_button_frame = tk.Frame(popup, bg="white")
        bottom_button_frame.pack(fill=tk.X, side=tk.BOTTOM, anchor="e")
    
        calculate_btn = tk.Button(bottom_button_frame, text="Calculate All", command=calc_all,
                                   bg="#007bff", fg="white", font=("Arial", 10, "bold"))
        calculate_btn.pack(side=tk.RIGHT, padx=10, pady=5)
    
        total_btn = tk.Button(bottom_button_frame, text="Total", command=show_grand_total,
                              bg="#28a745", fg="white", font=("Arial", 10, "bold"))
        total_btn.pack(side=tk.RIGHT, padx=10, pady=5)

        
  


    # ---------- Summary Frame (Buttons + Entry Fields) ----------
    summary_frame = tk.Frame(root, bg="#2e3f50")
    summary_frame.pack(fill="x", padx=10, pady=(5, 10))
    
    buttons = [
        ("ENCLOSURE FABRICATION", open_enclosure),
        ("WIRING + ACCESSORIES", open_wiring),
        ("BUS BAR", open_busbar),
        ("TRANSPORTATION", None),
        ("LABOUR", None)
    ]
    
    entry_vars = []
    
    for i, (text, cmd) in enumerate(buttons):
        # Create Button
        btn = ttk.Button(summary_frame, text=text, width=22, command=cmd if cmd else lambda: None)
        btn.grid(row=0, column=i*2, padx=5, pady=5, sticky="ew")
    
        # Create associated Entry
        entry = ttk.Entry(summary_frame, width=22)

        entry.grid(row=0, column=i*2 + 1, padx=5, pady=5)
        entry_vars.append(entry)
    
    # Optional: expand button columns
    for i in range(len(buttons) * 2):
        summary_frame.grid_columnconfigure(i, weight=1)
        
        # --- LP GROSS TOTAL Label ---
        tk.Label(summary_frame, text="LP GROSS TOTAL:", font=("Segoe UI", 12, "bold"),
                 bg="#2c3e50", fg="white").grid(row=1, column=0, sticky="e", padx=10, pady=5)
        
        tk.Label(summary_frame, textvariable=lp_total_var, font=("Segoe UI", 12,"bold"),
                 bg="white", fg="black", width=15, anchor="w", relief="solid").grid(row=1, column=1, sticky="w", padx=5, pady=5)
        
        # --- NET DISCOUNT Label ---
        tk.Label(summary_frame, text="NET DISC:", font=("Segoe UI", 12, "bold"),
                 bg="#2c3e50", fg="white").grid(row=1, column=2, sticky="e", padx=10, pady=5)
        
        net_discount_var = tk.StringVar(value="0.00")  # Declare this before if not already
        
        tk.Label(summary_frame, textvariable=net_discount_var, font=("Segoe UI", 12,"bold"),
                 bg="white", fg="black", width=15, anchor="w", relief="solid").grid(row=1, column=3, sticky="w", padx=5, pady=5)
          
        generate_btn = ttk.Button(summary_frame, text="GENERATE BILL", width=20)
        generate_btn.grid(row=1, column=8, padx=10)

        cancel_btn = ttk.Button(summary_frame, text="CANCEL", width=20)
        cancel_btn.grid(row=1, column=9, padx=10)


    root.mainloop()

Billing_Entry_UI()
 
 
 

In [113]:
import tkinter as tk

# Sample headers and data for demonstration
headers = ["", "MM", "Length", "Width", "MM*L*W*7.85", "Kg", "Description", "Kg/(L*W)"]
enclosure_data = [
    {"MM": "2", "Length": "200", "Width": "150", "Kg": "4.5", "Description": "Item A"},
    {"MM": "1.5", "Length": "100", "Width": "100", "Kg": "2.2", "Description": "Item B"},
    {"MM": "3", "Length": "300", "Width": "200", "Kg": "10.0", "Description": "Item C"},
]

root = tk.Tk()
popup = tk.Frame(root, bg="white")
popup.pack(fill="both", expand=True)

entry_matrix1 = []
checkbox_vars = []

def format_decimal(value):
    return "{:.10f}".format(value).rstrip("0").rstrip(".") if "." in "{:.10f}".format(value) else "{:.10f}".format(value)

def update_kg_per_sqmm(row_idx):
    try:
        kg = float(entry_matrix1[row_idx][5].get() or 0)
        length = float(entry_matrix1[row_idx][2].get() or 0)
        width = float(entry_matrix1[row_idx][3].get() or 0)

        if length > 0 and width > 0:
            result = kg / (length * width)
            formatted_result = format_decimal(result)
        else:
            formatted_result = "0.0000000000"

        entry_matrix1[row_idx][7].config(state='normal')
        entry_matrix1[row_idx][7].delete(0, tk.END)
        entry_matrix1[row_idx][7].insert(0, formatted_result)
        entry_matrix1[row_idx][7].config(state='readonly', disabledforeground="black")

    except Exception:
        entry_matrix1[row_idx][7].config(state='normal')
        entry_matrix1[row_idx][7].delete(0, tk.END)
        entry_matrix1[row_idx][7].insert(0, "Error")
        entry_matrix1[row_idx][7].config(state='readonly', disabledforeground="red")

popup.grid_columnconfigure(0, minsize=30)

# Header row
for col, text in enumerate(headers):
    lbl = tk.Label(popup, text=text if text else "Calculated",
                   font=('times new roman', 12, "bold"),
                   bg="#A1AEB1", fg="black", width=14, borderwidth=1, relief="solid")
    lbl.grid(row=0, column=col, padx=1, pady=8, sticky="nsew")

# Data rows
for row_index, data in enumerate(enclosure_data):
    row_entries = []
    var = tk.IntVar()
    checkbox_vars.append(var)

    # Checkbox
    chk = tk.Checkbutton(popup, variable=var, bg="white", padx=0, pady=0, takefocus=0)
    chk.grid(row=row_index + 1, column=0, padx=1, pady=3, sticky="w")
    row_entries.append(None)

    for col_index, header in enumerate(headers[1:], start=1):
        ent = tk.Entry(popup, font=('times new roman', 12), width=14,
                       borderwidth=1, relief="solid", justify='center')
        ent.grid(row=row_index + 1, column=col_index, padx=1, pady=3, sticky="nsew")

        if header in data:
            ent.insert(0, data[header])
        elif col_index == 4:  # MM*L*W*0.00000785
            mm = float(data.get("MM", 0))
            length = float(data.get("Length", 0))
            width = float(data.get("Width", 0))
            total = round(mm * length * width * 0.00000785, 2)
            ent.insert(0, total)
            ent.config(state='readonly', disabledforeground="black")
        elif col_index == 7:  # Kg / (L * W)
            kg = float(data.get("Kg", 0))
            length = float(data.get("Length", 0))
            width = float(data.get("Width", 0))
            if length > 0 and width > 0:
                result = kg / (length * width)
                formatted_result = format_decimal(result)
            else:
                formatted_result = "0.0000000000"
            ent.insert(0, formatted_result)
            ent.config(state='readonly', disabledforeground="black")

        if header in ["Length", "Width", "Kg"]:
            ent.bind("<FocusOut>", lambda e, r=row_index: update_kg_per_sqmm(r))
            ent.bind("<Return>", lambda e, r=row_index: update_kg_per_sqmm(r))
            ent.bind("<Tab>", lambda e, r=row_index: update_kg_per_sqmm(r))

        # Optional row highlight
        if row_index in [0, 2, 4, 6]:
            ent.config(bg="#FF7F7F")

        row_entries.append(ent)

    entry_matrix1.append(row_entries)

root.mainloop()
